<center>

## **Exploring Socio-Economic Trends in England and Wales: A Visual Analytics Approach**

</center>

### Table of Contents

[Environment Setup and Library Imports](#environment-setup-and-library-imports)

[Part 1: Load Census Data, Align Geographies, and Integrate Features](#section-1-load-census-data-align-geographies-and-integrate-features)  
  [1.1: Load and Inspect 2011 Census Datasets](#subsection-11-load-and-inspect-2011-census-datasets)  
  [1.2: Combine Multiple 2011 Census Datasets into a Unified Table](#subsection-12-combine-multiple-2011-census-datasets-into-a-unified-table)  
  [1.3: Standardize Column Names and Validate Missing Values](#subsection-13-standardize-column-names-and-validate-missing-values)  
  [1.4: Load and Inspect 2021 Census Datasets](#subsection-14-load-and-inspect-2021-census-datasets)  
  [1.5: Detect Geography Code Mismatches Between 2011 and 2021 Datasets](#subsection-15-detect-geography-code-mismatches-between-2011-and-2021-datasets)  
  [1.6: Align and Rename Accommodation Type Datasets to a 2011 Geography Framework](#subsection-16-align-and-rename-accommodation-type-datasets-to-a-2011-geography-framework)  
  [1.7: Align and Rename Tenure Datasets to a 2011 Geography Framework](#subsection-17-align-and-rename-tenure-datasets-to-a-2011-geography-framework)  
  [1.8: Align Shared Features for Accommodation Type (2011 and 2021)](#subsection-18-align-shared-features-for-accommodation-type-2011-and-2021)  
  [1.9: Align Shared Features for Tenure (2011 and 2021)](#subsection-19-align-shared-features-for-tenure-2011-and-2021)  

[Part 2: Bayesian Confirmatory Factor Analysis of Housing-Related Latent Constructs](#section-2-bayesian-confirmatory-factor-analysis-of-housing-related-latent-constructs)  
  [2.1: Estimate Latent Socio-Economic Constructs using Bayesian Confirmatory Factor Analysis](#subsection-21-estimate-latent-socio-economic-constructs-using-bayesian-confirmatory-factor-analysis)  
  [2.2: Reshape Latent Estimates Output Dataframe into Long Format for Tableau Visualization](#subsection-22-reshape-latent-estimates-output-dataframe-into-long-format-for-tableau-visualization)  
  [2.3: Bayesian CFA of Accommodation Structure (2011 vs 2021 Modeling)](#subsection-23-bayesian-cfa-of-accommodation-structure-2011-vs-2021-modeling)  
  [2.4: Pivot Accommodation Latent Scores into Year-Wide Format for 2011–2021 Comparison](#subsection-24-pivot-accommodation-latent-scores-into-year-wide-format-for-2011–2021-comparison)  
  [2.5: Bayesian CFA of Tenure Structure (2011 vs 2021 Modeling)](#subsection-25-bayesian-cfa-of-tenure-structure-2011-vs-2021-modeling)  
  [2.6: Pivot Tenure Latent Scores into Year-Wide Format for 2011–2021 Comparison](#subsection-26-pivot-tenure-latent-scores-into-year-wide-format-for-2011–2021-comparison)  

[Part 3: Dimensionality Reduction and Clustering of Socio-Economic Patterns](#section-3-dimensionality-reduction-and-clustering-of-socio-economic-patterns)  
  [3.1: PCA –– t-SNE Projection of Observed Socio-Economic Indicators (2011)](#subsection-31-pca-––-t-sne-projection-of-observed-socioeconomic-indicators-2011)  
  [3.2: UMAP Projection of Latent Socio-Economic Dimensions (2011)](#subsection-32-umap-projection-of-latent-socio-economic-dimensions-2011)  
  [3.3: UMAP Projection and Clustering of Accommodation Features (2011 vs 2021)](#subsection-33-umap-projection-and-clustering-of-accommodation-features-2011-vs-2021)  
  [3.4: UMAP Projection and Clustering of Tenure Features (2011 vs 2021)](#subsection-34-umap-projection-and-clustering-of-tenure-features-2011-vs-2021)  


<a id="environment-setup-and-library-imports"></a>
### Environment Setup and Library Imports

We ensured that all required packages and runtime libraries were available to support Bayesian modeling, clustering, and dimensionality reduction workflows for the analysis pipeline.

After checking for and installing necessary Python dependencies, we imported a structured suite of modules categorized as follows:

1) Bayesian Modeling and Diagnostics: pymc was used to specify and fit probabilistic graphical models, while arviz supported posterior analysis and convergence diagnostics. Compiler-level outputs from PyTensor were suppressed to improve runtime readability.

2) Data Handling and Core Utilities: pandas and numpy facilitated tabular data manipulation and numerical array computation. Additional utilities such as warnings, logging, unicodedata, and re supported silent execution and robust string handling. Static typing hints were imported from typing, and SequenceMatcher from difflib enabled pairwise string comparison.

3) Text Vectorization and Similarity: TfidfVectorizer and cosine_similarity from scikit-learn were used to convert text into vector space and calculate similarity between feature labels or geographic names.

4) Dimensionality Reduction and Clustering: StandardScaler, MinMaxScaler, PCA, TSNE, and UMAP supported feature scaling and projection for both linear and non-linear embeddings. KMeans was used for unsupervised clustering on reduced embeddings.

5) Notebook Display Utilities: IPython.display.display provided formatted table previews in Jupyter environments. Logging verbosity for PyMC and PyTensor was reduced to error-level only for clarity.

In [1]:
# Configure environment
import os, sys, warnings, logging, contextlib, io, subprocess
from contextlib import redirect_stdout, redirect_stderr

os.environ["PYMC_PROGRESSBAR"] = "0"  
os.environ["PYTENSOR_FLAGS"] = "cxx="       

warnings.filterwarnings("ignore", module="pytensor.configdefaults")
warnings.filterwarnings("ignore", category=FutureWarning)

warnings.filterwarnings("ignore", category=UserWarning)
for name in ["pytensor", "pytensor.configdefaults", "pymc"]:
    logging.getLogger(name).setLevel(logging.ERROR)

# Define required Python packages
pkgs = {
    'pymc': 'pymc',                                         # Probabilistic modeling
    'arviz': 'arviz',                                       # Bayesian diagnostics and visualization
    'pandas': 'pandas',                                     # Data manipulation
    'numpy': 'numpy',                                       # Numerical computing
    'scikit-learn': 'sklearn',                              # Machine learning and preprocessing
    'umap-learn': 'umap',                                   # Dimensionality reduction
    'ipython': 'IPython'                                    # Interactive display support
}

# Install packages if not already available in the current environment
for pip_name, import_name in pkgs.items():
    try:
        __import__(import_name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])                  

# Import Bayesian modeling libraries 
with contextlib.redirect_stderr(io.StringIO()):
    import pymc as pm                                       # Implements probabilistic models
    import arviz as az                                      # Visualizes Bayesian inference diagnostics

# General Utilities
import re                                                   # Handles regular expression-based string processing
import unicodedata                                          # Normalizes and inspects Unicode text representations
from typing import Dict, List, Optional, Set                # Defines type hints for static code checking
from difflib import SequenceMatcher                         # Computes similarity ratios between character sequences

# Data Processing
import pandas as pd                                         # Provides data structures and tools for data manipulation
import numpy as np                                          # Supports efficient numerical computations with arrays

# Set global random seed for reproducibility
np.random.seed(42)                                          
rng = np.random.default_rng(42)                            

# Text Vectorization and Similarity
from sklearn.feature_extraction.text import TfidfVectorizer # Converts text into TF-IDF feature vectors
from sklearn.metrics.pairwise import cosine_similarity      # Computes cosine similarity between vectorized texts

# Dimensionality Reduction
from sklearn.preprocessing import StandardScaler            # Standardizes features to zero mean and unit variance
from sklearn.preprocessing import MinMaxScaler              # Scales features to a specified range
from sklearn.decomposition import PCA                       # Reduces dimensionality using principal component analysis
from sklearn.manifold import TSNE                           # Projects high-dimensional data into 2D/3D using t-SNE
import umap.umap_ as umap                                   # Performs non-linear dimensionality reduction with UMAP

# Clustering
from sklearn.cluster import KMeans                          # Performs unsupervised clustering using KMeans algorithm

# Display Utilities
from IPython.display import display                         # Enables displays of DataFrames

<a id="section-1-load-census-data-align-geographies-and-integrate-features"></a>
### Part 1: Load Census Data, Align Geographies, and Integrate Features

In this part, we systematically processed census datasets from 2011 and 2021, covering accommodation type, tenure, household size and occupancy rating, and car availability.

We began by loading and inspecting datasets for both years (1.1–1.2), then detected and resolved geography code mismatches across years (1.3–1.4). We standardized feature column names (1.5) and applied alignment and renaming workflows for individual datasets (1.6–1.7). Finally, we identified shared features between 2011 and 2021 datasets and vertically stacked them into combined DataFrames for accommodation type and tenure (1.8–1.9), ensuring consistent schema and structure for cross-year comparison.

<a id="subsection-11-load-and-inspect-2011-census-datasets"></a>
#### 1.1: Load and Inspect 2011 Census Datasets

In the first step, we defined a reusable function to load and inspect census datasets from the year 2011.

We applied this function to four datasets: accommodation type, tenure, household size and occupancy rating, and car or van availability. For each dataset, we printed its dimensions, previewed the first few rows, and calculated missing value statistics to assess data completeness.

In [2]:
# Step 1: Define a function to load and inspect a census dataset from Excel or CSV
def load_excel_dataset(file_path, dataset_name, year_suffix):
    
    # Check file extension and load the dataset accordingly
    if file_path.endswith(".csv"):
        df_2011 = pd.read_csv(file_path)
    else:
        df_2011 = pd.read_excel(file_path)

    # Print dataset dimensions and display the first few rows
    print(f"\n[{dataset_name}_2011] Loaded shape: {df_2011.shape}\n")
    display(df_2011.head().style.hide(axis="index"))

    # Step 2: Check for missing values in the dataset
    total_missing = df_2011.isnull().sum().sum()
    print(f"\n[{dataset_name}_2011] Total missing values: {total_missing}")

    # If missing values exist, show the percentage per column
    if total_missing > 0:
        missing_by_column = df_2011.isnull().mean().round(4) * 100
        print(f"[{dataset_name}_2011] Missing value percentage by column:")
        print(missing_by_column[missing_by_column > 0].round(2).astype(str) + '%')
    else:
        print(f"[{dataset_name}_2011] No missing values found.\n")

    return df_2011

# Step 3: Define file paths for census 2011 datasets
file_accommodation_2011 = "QS402EW_accommodation_type_census2011.csv"
file_tenure_2011 = "QS405EW_tenure_census2011.csv"
file_household_size_2011 = "QS412EW_household_size_occupancy_rating_2011.csv"
file_car_availability_2011 = "QS416EW_car_or_van_availability_2011.csv"

# Step 4: Load each 2011 dataset
df_accommodation_2011 = load_excel_dataset(file_accommodation_2011, "Accommodation Type", "2011")
df_tenure_2011 = load_excel_dataset(file_tenure_2011, "Tenure", "2011")
df_household_size_2011 = load_excel_dataset(file_household_size_2011, "Household Size & Occupancy", "2011")
df_car_availability_2011 = load_excel_dataset(file_car_availability_2011, "Car or Van Availability", "2011")


[Accommodation Type_2011] Loaded shape: (348, 16)



date,geography,geography code,Rural Urban,Dwelling Type: All categories: Accommodation type; measures: Value,Dwelling Type: Unshared dwelling: Total; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Total; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Detached; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Semi-detached; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Terraced (including end-terrace); measures: Value,"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Total; measures: Value","Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Purpose-built block of flats or tenement; measures: Value","Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits); measures: Value","Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: In commercial building; measures: Value",Dwelling Type: Unshared dwelling: Caravan or other mobile or temporary structure; measures: Value,Dwelling Type: Shared dwelling; measures: Value
2011,Darlington,E06000005,Total,46670,46552,40423,8209,17835,14379,6099,5053,778,268,30,118
2011,County Durham,E06000047,Total,223803,223755,211091,43114,83225,84752,12436,9941,1306,1189,228,48
2011,Hartlepool,E06000001,Total,40434,40412,35203,6822,13448,14933,5121,4489,330,302,88,22
2011,Middlesbrough,E06000002,Total,57203,57070,49724,8001,23394,18329,7325,6389,589,347,21,133
2011,Northumberland,E06000057,Total,138534,138500,123592,35120,50127,38345,14676,11602,2142,932,232,34



[Accommodation Type_2011] Total missing values: 0
[Accommodation Type_2011] No missing values found.


[Tenure_2011] Loaded shape: (348, 18)



date,geography,geography code,Rural Urban,Tenure: All categories: Tenure; measures: Value,Tenure: Owned: Total; measures: Value,Tenure: Owned: Owned outright; measures: Value,Tenure: Owned: Owned with a mortgage or loan; measures: Value,Tenure: Shared ownership (part owned and part rented); measures: Value,Tenure: Social rented: Total; measures: Value,Tenure: Social rented: Rented from council (Local Authority); measures: Value,Tenure: Social rented: Other social rented; measures: Value,Tenure: Private rented: Total; measures: Value,Tenure: Private rented: Private landlord or letting agency; measures: Value,Tenure: Private rented: Employer of a household member; measures: Value,Tenure: Private rented: Relative or friend of household member; measures: Value,Tenure: Private rented: Other; measures: Value,Tenure: Living rent free; measures: Value
2011,Darlington,E06000005,Total,46670,30289,13810,16479,178,7228,5238,1990,8513,7758,112,546,97,462
2011,County Durham,E06000047,Total,223803,147347,69435,77912,488,45004,28559,16445,28142,25498,185,2121,338,2822
2011,Hartlepool,E06000001,Total,40434,24194,11090,13104,200,9515,4016,5499,5971,5525,26,349,71,554
2011,Middlesbrough,E06000002,Total,57203,32723,15200,17523,336,13654,6049,7605,9509,8648,98,570,193,981
2011,Northumberland,E06000057,Total,138534,91207,46086,45121,510,25841,14820,11021,18417,16225,434,1388,370,2559



[Tenure_2011] Total missing values: 0
[Tenure_2011] No missing values found.


[Household Size & Occupancy_2011] Loaded shape: (348, 10)



date,geography,geography code,Rural Urban,Occupancy Rating: All categories: Occupancy rating bedrooms; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of +2 or more; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of +1; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of 0; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of -1; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of -2 or less; measures: Value
2011,Darlington,E06000005,Total,46670,15682,18270,11450,1149,119
2011,County Durham,E06000047,Total,223803,77389,92840,48450,4599,525
2011,Hartlepool,E06000001,Total,40434,14261,15525,9576,947,125
2011,Middlesbrough,E06000002,Total,57203,18686,20890,15031,2246,350
2011,Northumberland,E06000057,Total,138534,54773,52023,28925,2555,258



[Household Size & Occupancy_2011] Total missing values: 0
[Household Size & Occupancy_2011] No missing values found.


[Car or Van Availability_2011] Loaded shape: (348, 11)



date,geography,geography code,Rural Urban,Cars: All categories: Car or van availability; measures: Value,Cars: No cars or vans in household; measures: Value,Cars: 1 car or van in household; measures: Value,Cars: 2 cars or vans in household; measures: Value,Cars: 3 cars or vans in household; measures: Value,Cars: 4 or more cars or vans in household; measures: Value,Cars: sum of All cars or vans in the area; measures: Value
2011,Darlington,E06000005,Total,46670,13052,20682,10450,1962,524,49794
2011,County Durham,E06000047,Total,223803,60926,96086,52740,10775,3276,248586
2011,Hartlepool,E06000001,Total,40434,14268,16573,7662,1535,396,38269
2011,Middlesbrough,E06000002,Total,57203,21488,22963,10207,1945,600,51821
2011,Northumberland,E06000057,Total,138534,30543,60875,36916,7671,2529,168974



[Car or Van Availability_2011] Total missing values: 0
[Car or Van Availability_2011] No missing values found.



<a id="subsection-12-combine-multiple-2011-census-datasets-into-a-unified-table"></a>
#### 1.2: Combine Multiple 2011 Census Datasets into a Unified Table

In this step, we defined a function to vertically concatenate four 2011 census datasets—accommodation type, tenure, household size and occupancy, and car availability—based on consistent identifier columns.

We verified that the first four ID columns were identical across datasets, and if so, merged all feature columns into a single DataFrame. The resulting dataset was previewed and summary statistics such as row count and the number of unique geography codes were printed.

In [3]:
# Step 1: Define a function to combine four census datasets from 2011 if their ID columns match
def prepare_2011_combined_dataset():

    # Check whether the first four columns (ID columns) are identical across all datasets
    if (
        df_accommodation_2011.iloc[:, :4].equals(df_tenure_2011.iloc[:, :4])
        and df_accommodation_2011.iloc[:, :4].equals(df_household_size_2011.iloc[:, :4])
        and df_accommodation_2011.iloc[:, :4].equals(df_car_availability_2011.iloc[:, :4])
    ):
        
        # Concatenate ID columns and feature columns from all datasets along columns
        df_combined_2011 = pd.concat(
            [
                df_accommodation_2011.iloc[:, :4],
                df_accommodation_2011.iloc[:, 4:],
                df_tenure_2011.iloc[:, 4:],
                df_household_size_2011.iloc[:, 4:],
                df_car_availability_2011.iloc[:, 4:]
            ],
            axis=1
        )

        # Display the first few rows of the combined dataset
        display(df_combined_2011.head().style.hide(axis="index"))

        # Print summary statistics
        print("\nTotal number of rows in the combined 2011 dataset:", len(df_combined_2011))
        print("Total Number of unique geography codes:", df_combined_2011["geography code"].nunique())

        return df_combined_2011

    # Raise an error if the ID columns are not consistent across datasets
    else:
        raise ValueError("ID columns (first 4) are not consistent across 2011 datasets.")

# Step 2: Run the function and store the result as the combined 2011 dataset
df_combined_2011 = prepare_2011_combined_dataset()

# Step 3: Extract the list of ID columns from the combined dataset
id_cols = df_combined_2011.columns[:4].tolist()

date,geography,geography code,Rural Urban,Dwelling Type: All categories: Accommodation type; measures: Value,Dwelling Type: Unshared dwelling: Total; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Total; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Detached; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Semi-detached; measures: Value,Dwelling Type: Unshared dwelling: Whole house or bungalow: Terraced (including end-terrace); measures: Value,"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Total; measures: Value","Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Purpose-built block of flats or tenement; measures: Value","Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits); measures: Value","Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: In commercial building; measures: Value",Dwelling Type: Unshared dwelling: Caravan or other mobile or temporary structure; measures: Value,Dwelling Type: Shared dwelling; measures: Value,Tenure: All categories: Tenure; measures: Value,Tenure: Owned: Total; measures: Value,Tenure: Owned: Owned outright; measures: Value,Tenure: Owned: Owned with a mortgage or loan; measures: Value,Tenure: Shared ownership (part owned and part rented); measures: Value,Tenure: Social rented: Total; measures: Value,Tenure: Social rented: Rented from council (Local Authority); measures: Value,Tenure: Social rented: Other social rented; measures: Value,Tenure: Private rented: Total; measures: Value,Tenure: Private rented: Private landlord or letting agency; measures: Value,Tenure: Private rented: Employer of a household member; measures: Value,Tenure: Private rented: Relative or friend of household member; measures: Value,Tenure: Private rented: Other; measures: Value,Tenure: Living rent free; measures: Value,Occupancy Rating: All categories: Occupancy rating bedrooms; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of +2 or more; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of +1; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of 0; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of -1; measures: Value,Occupancy Rating: Occupancy rating (bedrooms) of -2 or less; measures: Value,Cars: All categories: Car or van availability; measures: Value,Cars: No cars or vans in household; measures: Value,Cars: 1 car or van in household; measures: Value,Cars: 2 cars or vans in household; measures: Value,Cars: 3 cars or vans in household; measures: Value,Cars: 4 or more cars or vans in household; measures: Value,Cars: sum of All cars or vans in the area; measures: Value
2011,Darlington,E06000005,Total,46670,46552,40423,8209,17835,14379,6099,5053,778,268,30,118,46670,30289,13810,16479,178,7228,5238,1990,8513,7758,112,546,97,462,46670,15682,18270,11450,1149,119,46670,13052,20682,10450,1962,524,49794
2011,County Durham,E06000047,Total,223803,223755,211091,43114,83225,84752,12436,9941,1306,1189,228,48,223803,147347,69435,77912,488,45004,28559,16445,28142,25498,185,2121,338,2822,223803,77389,92840,48450,4599,525,223803,60926,96086,52740,10775,3276,248586
2011,Hartlepool,E06000001,Total,40434,40412,35203,6822,13448,14933,5121,4489,330,302,88,22,40434,24194,11090,13104,200,9515,4016,5499,5971,5525,26,349,71,554,40434,14261,15525,9576,947,125,40434,14268,16573,7662,1535,396,38269
2011,Middlesbrough,E06000002,Total,57203,57070,49724,8001,23394,18329,7325,6389,589,347,21,133,57203,32723,15200,17523,336,13654,6049,7605,9509,8648,98,570,193,981,57203,18686,20890,15031,2246,350,57203,21488,22963,10207,1945,600,51821
2011,Northumberland,E06000057,Total,138534,138500,123592,35120,50127,38345,14676,11602,2142,932,232,34,138534,91207,46086,45121,510,25841,14820,11021,18417,16225,434,1388,370,2559,138534,54773,52023,28925,2555,258,138534,30543,60875,36916,7671,2529,168974



Total number of rows in the combined 2011 dataset: 348
Total Number of unique geography codes: 348


<a id="subsection-13-standardize-column-names-and-validate-missing-values"></a>
#### 1.3: Standardize Column Names and Validate Missing Values

In this step, we applied a column renaming function to the combined 2011 census dataset, converting all feature names to a consistent format.

The function removed verbose prefixes and suffixes, applied manual corrections for special cases, and displayed a mapping table between original and renamed feature labels. After renaming, we previewed the cleaned dataset, checked for missing values, and reported missingness statistics at both global and column levels.

In [4]:
# Step 1: Define a function to rename feature columns to standardized snake_case format
def rename_columns_to_snake_case(df_final: pd.DataFrame, id_cols: list[str], df_label: str = "df_final") -> pd.DataFrame:

    # Define a helper function to slugify a string by lowercasing and replacing non-alphanumerics
    def slugify(text):
        text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("ascii")
        text = re.sub(r"[^0-9a-zA-Z]+", "_", text)
        return text.lower().strip("_")

    # Compile regular expressions to clean known prefixes and suffixes from original column names
    prefix_pattern = re.compile(r"^(Dwelling Type|Tenure|Occupancy Rating|Cars):\s*", re.I)
    suffix_pattern = re.compile(r";\s*measures:\s*Value$", re.I)

    # Identify feature columns to rename (excluding ID columns)
    raw_feature_cols = [c for c in df_final.columns if c not in id_cols]

    # Clean and convert each feature column name to snake_case
    cleaned_feature_cols = [
        slugify(suffix_pattern.sub("", prefix_pattern.sub("", col)))
        for col in raw_feature_cols
    ]

    # Manual correction for Occupancy Rating
    manual_corrections = {
        "Occupancy Rating: Occupancy rating (bedrooms) of +1; measures: Value": "occupancy_rating_bedrooms_of_plus_1",
        "Occupancy Rating: Occupancy rating (bedrooms) of -1; measures: Value": "occupancy_rating_bedrooms_of_minus_1",
        "Occupancy Rating: Occupancy rating (bedrooms) of +2 or more; measures: Value": "occupancy_rating_bedrooms_of_plus_2_or_more",
        "Occupancy Rating: Occupancy rating (bedrooms) of -2 or less; measures: Value": "occupancy_rating_bedrooms_of_minus_2_or_less"
    }

    # Apply manual corrections
    for i, col in enumerate(raw_feature_cols):
        if col in manual_corrections:
            cleaned_feature_cols[i] = manual_corrections[col]

    # Create a mapping from original names to cleaned names
    rename_map = dict(zip(raw_feature_cols, cleaned_feature_cols))

    # Step 2: Display a mapping table between original and renamed columns
    rename_df = pd.DataFrame({
        "Original Column Name": raw_feature_cols,
        "Renamed Column": cleaned_feature_cols
    })

    styled_rename_df = (
        rename_df.style
        .set_table_styles([
            {"selector": "table", "props": [("border", "1px solid black"), ("border-collapse", "collapse")]},
            {"selector": "th", "props": [
                ("text-align", "left"),
                ("border-top", "1px solid black"),
                ("border-bottom", "1px solid black"),
                ("border-right", "1px solid black"),
                ("border-left", "1px solid black")
            ]},
            {"selector": "td", "props": [
                ("text-align", "left"),
                ("border-right", "1px solid black"),
                ("border-left", "1px solid black"),
                ("border-bottom", "1px solid black")
            ]}
        ])
        .set_properties(**{"text-align": "left"})
        .hide(axis="index")
    )

    print("\nOverview of Original Feature Names and Corresponding Simplified Identifiers:\n")
    display(styled_rename_df)

    # Step 3: Apply the renaming map to the DataFrame
    df_final = df_final.rename(columns=rename_map)

    # Step 4: Preview the renamed dataset and display basic shape
    print(f"\n[{df_label}] shape after renaming: {df_final.shape}\n")
    display(df_final.head().style.hide(axis="index"))

    # Step 5: Check for missing values across the entire DataFrame
    total_missing = df_final.isnull().sum().sum()
    print(f"\n[{df_label}] Total missing values: {total_missing}")

    # Step 6: If missing values exist, show their percentage
    if total_missing > 0:
        missing_by_column = df_final.isnull().mean().round(4) * 100
        print(f"[{df_label}] Missing value percentage by column:")
        print(missing_by_column[missing_by_column > 0].round(2).astype(str) + '%')
    else:
        print(f"[{df_label}] No missing values found.")

    return df_final

# Step 7: Apply the column renaming function to the aligned 2011 census dataset
df_combined_2011_final = rename_columns_to_snake_case(
    df_combined_2011,
    id_cols,
    df_label="df_combined_2011_final"
)


Overview of Original Feature Names and Corresponding Simplified Identifiers:



Original Column Name,Renamed Column
Dwelling Type: All categories: Accommodation type; measures: Value,all_categories_accommodation_type
Dwelling Type: Unshared dwelling: Total; measures: Value,unshared_dwelling_total
Dwelling Type: Unshared dwelling: Whole house or bungalow: Total; measures: Value,unshared_dwelling_whole_house_or_bungalow_total
Dwelling Type: Unshared dwelling: Whole house or bungalow: Detached; measures: Value,unshared_dwelling_whole_house_or_bungalow_detached
Dwelling Type: Unshared dwelling: Whole house or bungalow: Semi-detached; measures: Value,unshared_dwelling_whole_house_or_bungalow_semi_detached
Dwelling Type: Unshared dwelling: Whole house or bungalow: Terraced (including end-terrace); measures: Value,unshared_dwelling_whole_house_or_bungalow_terraced_including_end_terrace
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Total; measures: Value",unshared_dwelling_flat_maisonette_or_apartment_total
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Purpose-built block of flats or tenement; measures: Value",unshared_dwelling_flat_maisonette_or_apartment_purpose_built_block_of_flats_or_tenement
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits); measures: Value",unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: In commercial building; measures: Value",unshared_dwelling_flat_maisonette_or_apartment_in_commercial_building



[df_combined_2011_final] shape after renaming: (348, 43)



date,geography,geography code,Rural Urban,all_categories_accommodation_type,unshared_dwelling_total,unshared_dwelling_whole_house_or_bungalow_total,unshared_dwelling_whole_house_or_bungalow_detached,unshared_dwelling_whole_house_or_bungalow_semi_detached,unshared_dwelling_whole_house_or_bungalow_terraced_including_end_terrace,unshared_dwelling_flat_maisonette_or_apartment_total,unshared_dwelling_flat_maisonette_or_apartment_purpose_built_block_of_flats_or_tenement,unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits,unshared_dwelling_flat_maisonette_or_apartment_in_commercial_building,unshared_dwelling_caravan_or_other_mobile_or_temporary_structure,shared_dwelling,all_categories_tenure,owned_total,owned_owned_outright,owned_owned_with_a_mortgage_or_loan,shared_ownership_part_owned_and_part_rented,social_rented_total,social_rented_rented_from_council_local_authority,social_rented_other_social_rented,private_rented_total,private_rented_private_landlord_or_letting_agency,private_rented_employer_of_a_household_member,private_rented_relative_or_friend_of_household_member,private_rented_other,living_rent_free,all_categories_occupancy_rating_bedrooms,occupancy_rating_bedrooms_of_plus_2_or_more,occupancy_rating_bedrooms_of_plus_1,occupancy_rating_bedrooms_of_0,occupancy_rating_bedrooms_of_minus_1,occupancy_rating_bedrooms_of_minus_2_or_less,all_categories_car_or_van_availability,no_cars_or_vans_in_household,1_car_or_van_in_household,2_cars_or_vans_in_household,3_cars_or_vans_in_household,4_or_more_cars_or_vans_in_household,sum_of_all_cars_or_vans_in_the_area
2011,Darlington,E06000005,Total,46670,46552,40423,8209,17835,14379,6099,5053,778,268,30,118,46670,30289,13810,16479,178,7228,5238,1990,8513,7758,112,546,97,462,46670,15682,18270,11450,1149,119,46670,13052,20682,10450,1962,524,49794
2011,County Durham,E06000047,Total,223803,223755,211091,43114,83225,84752,12436,9941,1306,1189,228,48,223803,147347,69435,77912,488,45004,28559,16445,28142,25498,185,2121,338,2822,223803,77389,92840,48450,4599,525,223803,60926,96086,52740,10775,3276,248586
2011,Hartlepool,E06000001,Total,40434,40412,35203,6822,13448,14933,5121,4489,330,302,88,22,40434,24194,11090,13104,200,9515,4016,5499,5971,5525,26,349,71,554,40434,14261,15525,9576,947,125,40434,14268,16573,7662,1535,396,38269
2011,Middlesbrough,E06000002,Total,57203,57070,49724,8001,23394,18329,7325,6389,589,347,21,133,57203,32723,15200,17523,336,13654,6049,7605,9509,8648,98,570,193,981,57203,18686,20890,15031,2246,350,57203,21488,22963,10207,1945,600,51821
2011,Northumberland,E06000057,Total,138534,138500,123592,35120,50127,38345,14676,11602,2142,932,232,34,138534,91207,46086,45121,510,25841,14820,11021,18417,16225,434,1388,370,2559,138534,54773,52023,28925,2555,258,138534,30543,60875,36916,7671,2529,168974



[df_combined_2011_final] Total missing values: 0
[df_combined_2011_final] No missing values found.


<a id="subsection-14-load-and-inspect-2021-census-datasets"></a>
#### 1.4: Load and Inspect 2021 Census Datasets

In this step, we reused the same data loading and inspection function to process census datasets from the year 2021.

We loaded and inspected two datasets: accommodation type and tenure. Similar to the previous subsection, the function output dataset dimensions, displayed sample rows, and reported missing value counts and proportions to facilitate quality checks.

In [5]:
# Step 1: Define a function to load and inspect a census dataset from Excel or CSV
def load_excel_dataset(file_path, dataset_name, year_suffix):
    
    # Check file extension and load the dataset accordingly
    if file_path.endswith(".csv"):
        df_2021 = pd.read_csv(file_path)
    else:
        df_2021 = pd.read_excel(file_path)

    # Print dataset dimensions and display the first few rows
    print(f"\n[{dataset_name}_2021] Loaded shape: {df_2021.shape}\n")
    display(df_2021.head().style.hide(axis="index"))

    # Step 2: Check for missing values in the dataset
    total_missing = df_2021.isnull().sum().sum()
    print(f"\n[{dataset_name}_2021] Total missing values: {total_missing}")

    # If missing values exist, show the percentage per column
    if total_missing > 0:
        missing_by_column = df_2021.isnull().mean().round(4) * 100
        print(f"[{dataset_name}_2021] Missing value percentage by column:")
        print(missing_by_column[missing_by_column > 0].round(2).astype(str) + '%')
    else:
        print(f"[{dataset_name}] No missing values found.\n")

    return df_2021

# Step 3: Define file paths for census 2021 datasets
file_accommodation_2021 = "TS044_accommodation_type_census2021.csv"
file_tenure_2021 = "TS054_tenure_census2021.csv"

# Step 4: Load each 2021 dataset
df_accommodation_2021 = load_excel_dataset(file_accommodation_2021, "Accommodation Type", "2021")
df_tenure_2021 = load_excel_dataset(file_tenure_2021, "Tenure", "2021")


[Accommodation Type_2021] Loaded shape: (331, 12)



date,geography,geography code,Accommodation type: Total: All households,Accommodation type: Detached,Accommodation type: Semi-detached,Accommodation type: Terraced,Accommodation type: In a purpose-built block of flats or tenement,"Accommodation type: Part of a converted or shared house, including bedsits","Accommodation type: Part of another converted building, for example, former school, church or warehouse","Accommodation type: In a commercial building, for example, in an office building, hotel or over a shop",Accommodation type: A caravan or other mobile or temporary structure
2021,Hartlepool,E06000001,40932,8086,14537,13469,3995,314,156,260,115
2021,Middlesbrough,E06000002,60262,10389,25485,16768,6457,647,157,308,51
2021,Redcar and Cleveland,E06000003,61637,12449,27942,15666,4005,791,312,388,84
2021,Stockton-on-Tees,E06000004,83756,22209,35494,18298,6444,723,175,334,79
2021,Darlington,E06000005,48920,9331,19148,13937,5271,743,172,240,78



[Accommodation Type_2021] Total missing values: 0
[Accommodation Type] No missing values found.


[Tenure_2021] Loaded shape: (331, 16)



date,geography,geography code,Tenure of household: Total: All households,Tenure of household: Owned,Tenure of household: Owned: Owns outright,Tenure of household: Owned: Owns with a mortgage or loan,Tenure of household: Shared ownership,Tenure of household: Shared ownership: Shared ownership,Tenure of household: Social rented,Tenure of household: Social rented: Rents from council or Local Authority,Tenure of household: Social rented: Other social rented,Tenure of household: Private rented,Tenure of household: Private rented: Private landlord or letting agency,Tenure of household: Private rented: Other private rented,Tenure of household: Lives rent free
2021,Hartlepool,E06000001,40929,23834,12761,11073,239,239,9594,3986,5608,7194,6338,856,68
2021,Middlesbrough,E06000002,60265,33137,17224,15913,330,330,14006,6773,7233,12677,11286,1391,115
2021,Redcar and Cleveland,E06000003,61638,39854,22649,17205,346,346,11649,5269,6380,9748,8410,1338,41
2021,Stockton-on-Tees,E06000004,83756,55412,27726,27686,385,385,13673,7204,6469,14207,12388,1819,79
2021,Darlington,E06000005,48915,30696,16222,14474,190,190,7931,5112,2819,10078,9035,1043,20



[Tenure_2021] Total missing values: 0
[Tenure] No missing values found.



<a id="subsection-15-detect-geography-code-mismatches-between-2011-and-2021-datasets"></a>
#### 1.5: Detect Geography Code Mismatches Between 2011 and 2021 Datasets

In this step, we defined functions to identify mismatched geography codes between paired census datasets from 2011 and 2021.

We first located geography name and code columns using flexible string matching, then compared the code sets across years for each dataset. Discrepancies were collected into a structured DataFrame showing which geography codes appeared in 2011 only, 2021 only, or both. We displayed the results in styled tables to support manual inspection of code mismatches for each dataset.

In [6]:
# Step 1: Define a function to locate geography and geography code columns in a DataFrame
def find_geo_cols(df):
    
    # Initialize placeholders for geography name and code columns
    geo_name, geo_code = None, None

    # Search for relevant column names by checking substrings
    for col in df.columns:
        low = col.lower()
        if geo_name is None and "geography" in low and "code" not in low:
            geo_name = col
        if geo_code is None and "geography" in low and "code" in low:
            geo_code = col

    # Raise an error if required columns are missing
    if geo_name is None or geo_code is None:
        raise ValueError("‘geography’ and/or ‘geography code’ column not found.")

    return geo_name, geo_code

# Step 2: Compare geography codes across two census datasets and identify mismatches
def compare_geographies(df11, df21, name):
    
    # Locate the geography name and code columns for each dataset
    g11, c11 = find_geo_cols(df11)
    g21, c21 = find_geo_cols(df21)

    # Build code-to-name mappings for both datasets
    map11 = dict(zip(df11[c11], df11[g11]))
    map21 = dict(zip(df21[c21], df21[g21]))

    # Identify geography codes that exist in one dataset but not the other
    only11 = set(map11) - set(map21)
    only21 = set(map21) - set(map11)

    # Construct a list of mismatch rows describing where codes are missing
    rows = (
        [
            {
                "Dataset": name,
                "Geography": map11[code],
                "Geography Code": code,
                "Present In": "2011 only",
            }
            for code in sorted(only11)
        ]
        + [
            {
                "Dataset": name,
                "Geography": map21[code],
                "Geography Code": code,
                "Present In": "2021 only",
            }
            for code in sorted(only21)
        ]
    )

    return rows

# Step 3: Build and display mismatch tables for a list of dataset pairs
def build_and_display_mismatch_tables(dataset_pairs):

    # Enable full display of all rows in the output
    pd.set_option("display.max_rows", None)

    # Initialize a dictionary to store mismatch DataFrames for each dataset
    mismatch_dfs = {}

    # Iterate over each dataset pair and compute mismatch rows
    for name, df11, df21 in dataset_pairs:
        mismatch_rows = compare_geographies(df11, df21, name)

        # Convert mismatch records to a styled DataFrame
        mismatch_df = (
            pd.DataFrame(mismatch_rows)
              .sort_values(["Present In", "Geography Code"])
              .reset_index(drop=True)
        )

        mismatch_dfs[name] = mismatch_df

        # Print and display the mismatch table with full formatting
        print(f"\nGeography Code Mismatches for Dataset: {name}\n")
        display(
            mismatch_df.style
              .set_table_styles([
                  {'selector': 'table', 'props': [('border', '1px solid black'), ('border-collapse', 'collapse')]},
                  {'selector': 'th', 'props': [
                      ('text-align', 'left'),
                      ('border-top', '1px solid black'),
                      ('border-bottom', '1px solid black'),
                      ('border-right', '1px solid black'),
                      ('border-left', '1px solid black')
                  ]},
                  {'selector': 'td', 'props': [
                      ('text-align', 'left'),
                      ('border-right', '1px solid black'),
                      ('border-left', '1px solid black'),
                      ('border-bottom', '1px solid black')
                  ]}
              ])
              .set_properties(**{'text-align': 'left'})
              .hide(axis="index")
        )

    return mismatch_dfs

# Step 4: Define dataset pairs to be compared for geographic consistency
dataset_pairs = [
    ("Accommodation Type", df_accommodation_2011, df_accommodation_2021),
    ("Tenure",             df_tenure_2011,        df_tenure_2021),
]

# Step 5: Run the comparison function and display mismatch tables
mismatch_dfs = build_and_display_mismatch_tables(dataset_pairs)


Geography Code Mismatches for Dataset: Accommodation Type



Dataset,Geography,Geography Code,Present In
Accommodation Type,Bournemouth,E06000028,2011 only
Accommodation Type,Poole,E06000029,2011 only
Accommodation Type,Aylesbury Vale,E07000004,2011 only
Accommodation Type,Chiltern,E07000005,2011 only
Accommodation Type,South Bucks,E07000006,2011 only
Accommodation Type,Wycombe,E07000007,2011 only
Accommodation Type,Christchurch,E07000048,2011 only
Accommodation Type,East Dorset,E07000049,2011 only
Accommodation Type,North Dorset,E07000050,2011 only
Accommodation Type,Purbeck,E07000051,2011 only



Geography Code Mismatches for Dataset: Tenure



Dataset,Geography,Geography Code,Present In
Tenure,Bournemouth,E06000028,2011 only
Tenure,Poole,E06000029,2011 only
Tenure,Aylesbury Vale,E07000004,2011 only
Tenure,Chiltern,E07000005,2011 only
Tenure,South Bucks,E07000006,2011 only
Tenure,Wycombe,E07000007,2011 only
Tenure,Christchurch,E07000048,2011 only
Tenure,East Dorset,E07000049,2011 only
Tenure,North Dorset,E07000050,2011 only
Tenure,Purbeck,E07000051,2011 only


<a id="subsection-16-align-and-rename-accommodation-type-datasets-to-a-2011-geography-framework"></a>
#### 1.6: Align and Rename Accommodation Type Datasets to a 2011 Geography Framework

In this step, we aligned the 2021 accommodation type dataset to the 2011 geographical framework using a combination of TF-IDF-based similarity, token subset matching, and string root comparison.

We handled one-to-many code splits using a manual mapping strategy and aggregated data by the target 2011 geography codes. After alignment, we renamed the columns of both the 2011 and 2021 accommodation datasets into snake_case format for consistency and previewed the processed outputs.

In [7]:
# Step 1: Define a function to align 2021 geographical codes to match 2011 codes
def align_2021_to_2011_geocodes(
    df_2021: pd.DataFrame,
    df_2011: pd.DataFrame,
    mismatch_df: pd.DataFrame,
    id_cols: List[str]
) -> pd.DataFrame:

    # Step 2: Identify feature columns (excluding ID columns)
    raw_feature_cols = df_2021.columns[4:]
    df_features_2021 = df_2021.copy()

    # Step 3: Separate unmatched 2021 and 2011 rows from the mismatch table
    new_rows = mismatch_df[mismatch_df["Present In"] == "2021 only"]
    old_rows = mismatch_df[mismatch_df["Present In"] == "2011 only"]

    # Step 4: Fit a TF-IDF model on 2011 geography names and transform them
    tfidf = TfidfVectorizer().fit(old_rows["Geography"])
    old_vecs = tfidf.transform(old_rows["Geography"])

    # Step 5: Find best-matching 2011 geography code for a given 2021 name using cosine similarity
    def best_match_tfidf(new_name: str, thr: float = 0.30):
        vec = tfidf.transform([new_name])
        sims = cosine_similarity(vec, old_vecs).flatten()
        idx = sims.argmax()
        return old_rows.iloc[idx]["Geography Code"] if sims[idx] >= thr else None

    # Step 6: Apply matching for all 2021-only codes
    merge_map = {}
    for ncode, nname in zip(new_rows["Geography Code"], new_rows["Geography"]):
        m = best_match_tfidf(nname)
        if m:
            merge_map[ncode] = m

    # Step 7: Token-based fallback matching
    def tokens(txt: str):
        return set(re.sub(r"[^a-z ]+", " ", txt.lower()).split())

    remaining = new_rows[~new_rows["Geography Code"].isin(merge_map)]
    for _, row_new in remaining.iterrows():
        ntoks = tokens(row_new["Geography"])
        for _, row_old in old_rows.iterrows():
            if ntoks.issubset(tokens(row_old["Geography"])):
                merge_map[row_new["Geography Code"]] = row_old["Geography Code"]
                break

    # Step 8: Define simplified root names for character similarity matching
    stop = {"north", "south", "east", "west", "county", "district", "unitary"}
    def root(name: str) -> str:
        words = [w for w in re.sub(r"[^a-z ]+", " ", name.lower()).split() if w not in stop]
        if words and words[-1].endswith("shire"):
            words[-1] = words[-1].removesuffix("shire")
        return " ".join(words)

    # Step 9: Fuzzy string matching using root names and SequenceMatcher
    remaining = new_rows[~new_rows["Geography Code"].isin(merge_map)]
    for _, row_new in remaining.iterrows():
        r_new = root(row_new["Geography"])
        best_code, best_sim = None, 0.0
        for _, row_old in old_rows.iterrows():
            sim = SequenceMatcher(None, r_new, root(row_old["Geography"])).ratio()
            if sim > best_sim:
                best_sim, best_code = sim, row_old["Geography Code"]
        if best_sim >= 0.80:
            merge_map[row_new["Geography Code"]] = best_code

    # Step 10: Fallback matching using highest TF-IDF similarity
    remaining = new_rows[~new_rows["Geography Code"].isin(merge_map)]
    if not remaining.empty:
        vec_rem = tfidf.transform(remaining["Geography"])
        sims = cosine_similarity(vec_rem, old_vecs)
        top_idx = sims.argmax(axis=1)
        for (ncode, _), j in zip(remaining.iterrows(), top_idx):
            merge_map[ncode] = old_rows.iloc[j]["Geography Code"]

    # Step 11: Handle one-to-many split by distributing values across multiple 2011 units
    remaining_new = new_rows[~new_rows["Geography Code"].isin(merge_map)]
    remaining_old = old_rows[~old_rows["Geography Code"].isin(merge_map.values())]
    if len(remaining_old) == 1 and not remaining_new.empty:
        target_code = remaining_old.iloc[0]["Geography Code"]
        merge_map.update({ncode: target_code for ncode in remaining_new["Geography Code"]})

    # Step 12: Define manual splits for complex code changes
    manual_split_map = {
        "E06000058": ["E06000028", "E06000029", "E07000048"],
        "E06000059": ["E07000049", "E07000050", "E07000051", "E07000052", "E07000053"],
        "E06000060": ["E07000004", "E07000005", "E07000006", "E07000007"],
        "E06000061": ["E07000150", "E07000152", "E07000153", "E07000156"],
        "E06000062": ["E07000151", "E07000154", "E07000155"],
        "E07000246": ["E07000190", "E07000191"],
        "E07000244": ["E07000205", "E07000206"],
        "E07000245": ["E07000201", "E07000204"]
    }

    # Step 13: Expand the 2021 data by duplicating rows with new 2011 codes
    geo_map_2011 = df_2011[["geography code", "geography"]].drop_duplicates().set_index("geography code")
    expanded_rows = []
    for _, row in df_features_2021.iterrows():
        gcode = row["geography code"]
        if gcode in manual_split_map:
            for tgt in manual_split_map[gcode]:
                new_row = row.copy()
                new_row["geography code"] = tgt
                new_row["geography"] = geo_map_2011.loc[tgt, "geography"]
                expanded_rows.append(new_row)
        else:
            new_row = row.copy()
            if gcode in merge_map:
                new_row["geography code"] = merge_map[gcode]
                new_row["geography"] = geo_map_2011.loc[merge_map[gcode], "geography"]
            expanded_rows.append(new_row)

    # Step 14: Filter and aggregate the expanded DataFrame by 2011 codes
    df_expanded = pd.DataFrame(expanded_rows)
    geo_codes_2011 = set(df_2011["geography code"])
    df_expanded = df_expanded[df_expanded["geography code"].isin(geo_codes_2011)]

    numeric_cols = (
        df_expanded.select_dtypes(include="number")
        .columns.difference(["date", "geography code"])
        .tolist()
    )
    df_sum = df_expanded.groupby("geography code", as_index=False)[numeric_cols].sum()
    df_date = df_expanded.groupby("geography code", as_index=False)["date"].first()
    df_geo = df_expanded.groupby("geography code", as_index=False)["geography"].first()

    # Step 15: Merge date and name back to the numeric summary
    df_2021_aligned = (
        df_sum
        .merge(df_date, on="geography code", how="left")
        .merge(df_geo, on="geography code", how="left")
    )

    # Step 16: Finalize column ordering and output
    if "geography" not in id_cols:
        id_cols.append("geography")
    final_cols = id_cols + list(raw_feature_cols)
    df_2021_aligned = df_2021_aligned[final_cols]

    return df_2021_aligned

# Step 17: Align and rename Accommodation Type Datasets using the defined function
print("\n>>> Processing: Accommodation Type (2021 Census)")
df_accommodation_2021_aligned = align_2021_to_2011_geocodes(
    df_2021 = df_accommodation_2021,
    df_2011 = df_accommodation_2011,
    mismatch_df = mismatch_dfs["Accommodation Type"][["Geography Code", "Geography", "Present In"]],
    id_cols = df_accommodation_2021.columns[:4].tolist()
)

print("\nGeographical codes in 'df_accommodation_2021_aligned' have been successfully aligned to the 2011 geographical framework.")

df_accommodation_2021_final = rename_columns_to_snake_case(
    df_accommodation_2021_aligned,
    id_cols,
    df_label="df_accommodation_2021_final"
)

print("\n" + "-" * 50)
print(">>> Processing: Accommodation Type (2011 Census)")

df_accommodation_2011_final = rename_columns_to_snake_case(
    df_accommodation_2011,
    id_cols,
    df_label="df_accommodation_2011_final"
)


>>> Processing: Accommodation Type (2021 Census)

Geographical codes in 'df_accommodation_2021_aligned' have been successfully aligned to the 2011 geographical framework.

Overview of Original Feature Names and Corresponding Simplified Identifiers:



Original Column Name,Renamed Column
Accommodation type: Total: All households,accommodation_type_total_all_households
Accommodation type: Detached,accommodation_type_detached
Accommodation type: Semi-detached,accommodation_type_semi_detached
Accommodation type: Terraced,accommodation_type_terraced
Accommodation type: In a purpose-built block of flats or tenement,accommodation_type_in_a_purpose_built_block_of_flats_or_tenement
"Accommodation type: Part of a converted or shared house, including bedsits",accommodation_type_part_of_a_converted_or_shared_house_including_bedsits
"Accommodation type: Part of another converted building, for example, former school, church or warehouse",accommodation_type_part_of_another_converted_building_for_example_former_school_church_or_warehouse
"Accommodation type: In a commercial building, for example, in an office building, hotel or over a shop",accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop
Accommodation type: A caravan or other mobile or temporary structure,accommodation_type_a_caravan_or_other_mobile_or_temporary_structure



[df_accommodation_2021_final] shape after renaming: (348, 12)



date,geography,geography code,accommodation_type_total_all_households,accommodation_type_detached,accommodation_type_semi_detached,accommodation_type_terraced,accommodation_type_in_a_purpose_built_block_of_flats_or_tenement,accommodation_type_part_of_a_converted_or_shared_house_including_bedsits,accommodation_type_part_of_another_converted_building_for_example_former_school_church_or_warehouse,accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop,accommodation_type_a_caravan_or_other_mobile_or_temporary_structure
2021,Hartlepool,E06000001,40932,8086,14537,13469,3995,314,156,260,115
2021,Middlesbrough,E06000002,60262,10389,25485,16768,6457,647,157,308,51
2021,Redcar and Cleveland,E06000003,61637,12449,27942,15666,4005,791,312,388,84
2021,Stockton-on-Tees,E06000004,83756,22209,35494,18298,6444,723,175,334,79
2021,Darlington,E06000005,48920,9331,19148,13937,5271,743,172,240,78



[df_accommodation_2021_final] Total missing values: 0
[df_accommodation_2021_final] No missing values found.

--------------------------------------------------
>>> Processing: Accommodation Type (2011 Census)

Overview of Original Feature Names and Corresponding Simplified Identifiers:



Original Column Name,Renamed Column
Dwelling Type: All categories: Accommodation type; measures: Value,all_categories_accommodation_type
Dwelling Type: Unshared dwelling: Total; measures: Value,unshared_dwelling_total
Dwelling Type: Unshared dwelling: Whole house or bungalow: Total; measures: Value,unshared_dwelling_whole_house_or_bungalow_total
Dwelling Type: Unshared dwelling: Whole house or bungalow: Detached; measures: Value,unshared_dwelling_whole_house_or_bungalow_detached
Dwelling Type: Unshared dwelling: Whole house or bungalow: Semi-detached; measures: Value,unshared_dwelling_whole_house_or_bungalow_semi_detached
Dwelling Type: Unshared dwelling: Whole house or bungalow: Terraced (including end-terrace); measures: Value,unshared_dwelling_whole_house_or_bungalow_terraced_including_end_terrace
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Total; measures: Value",unshared_dwelling_flat_maisonette_or_apartment_total
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Purpose-built block of flats or tenement; measures: Value",unshared_dwelling_flat_maisonette_or_apartment_purpose_built_block_of_flats_or_tenement
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits); measures: Value",unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits
"Dwelling Type: Unshared dwelling: Flat, maisonette or apartment: In commercial building; measures: Value",unshared_dwelling_flat_maisonette_or_apartment_in_commercial_building



[df_accommodation_2011_final] shape after renaming: (348, 16)



date,geography,geography code,Rural Urban,all_categories_accommodation_type,unshared_dwelling_total,unshared_dwelling_whole_house_or_bungalow_total,unshared_dwelling_whole_house_or_bungalow_detached,unshared_dwelling_whole_house_or_bungalow_semi_detached,unshared_dwelling_whole_house_or_bungalow_terraced_including_end_terrace,unshared_dwelling_flat_maisonette_or_apartment_total,unshared_dwelling_flat_maisonette_or_apartment_purpose_built_block_of_flats_or_tenement,unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits,unshared_dwelling_flat_maisonette_or_apartment_in_commercial_building,unshared_dwelling_caravan_or_other_mobile_or_temporary_structure,shared_dwelling
2011,Darlington,E06000005,Total,46670,46552,40423,8209,17835,14379,6099,5053,778,268,30,118
2011,County Durham,E06000047,Total,223803,223755,211091,43114,83225,84752,12436,9941,1306,1189,228,48
2011,Hartlepool,E06000001,Total,40434,40412,35203,6822,13448,14933,5121,4489,330,302,88,22
2011,Middlesbrough,E06000002,Total,57203,57070,49724,8001,23394,18329,7325,6389,589,347,21,133
2011,Northumberland,E06000057,Total,138534,138500,123592,35120,50127,38345,14676,11602,2142,932,232,34



[df_accommodation_2011_final] Total missing values: 0
[df_accommodation_2011_final] No missing values found.


<a id="subsection-17-align-and-rename-tenure-datasets-to-a-2011-geography-framework"></a>
#### 1.7: Align and Rename Tenure Datasets to a 2011 Geography Framework

In this step, we applied the same alignment workflow used for accommodation data to the 2021 tenure dataset, aligning its geography codes to match those from 2011.

After applying string similarity and rule-based mapping, we aggregated and reformatted the aligned dataset. The resulting 2021 data, along with the original 2011 tenure dataset, was renamed using a standardized snake_case format to ensure structural consistency across years.

In [8]:
# Step 1: Align the 2021 tenure dataset to 2011 geographical codes using the same procedure
print("\n>>> Processing: Tenure (2021 Census)")
df_tenure_2021_aligned = align_2021_to_2011_geocodes(
    df_2021 = df_tenure_2021,
    df_2011 = df_tenure_2011,
    mismatch_df = mismatch_dfs["Tenure"][["Geography Code", "Geography", "Present In"]],
    id_cols = df_tenure_2021.columns[:4].tolist()
)

# Step 2: Confirm alignment and show message
print("\nGeographical codes in 'df_tenure_2021_aligned' have been successfully aligned to the 2011 geographical framework.")

# Step 3: Standardize column naming for the aligned 2021 tenure dataset
df_tenure_2021_final = rename_columns_to_snake_case(
    df_tenure_2021_aligned,
    id_cols,
    df_label="df_tenure_2021_final"
)

# Step 4: Print visual separator and begin processing of 2011 tenure data
print("\n" + "-" * 50)
print(">>> Processing: Tenure (2011 Census)")

# Step 5: Rename columns in the original 2011 tenure dataset
df_tenure_2011_final = rename_columns_to_snake_case(
    df_tenure_2011,
    id_cols,
    df_label="df_tenure_2011_final"
)


>>> Processing: Tenure (2021 Census)

Geographical codes in 'df_tenure_2021_aligned' have been successfully aligned to the 2011 geographical framework.

Overview of Original Feature Names and Corresponding Simplified Identifiers:



Original Column Name,Renamed Column
Tenure of household: Total: All households,tenure_of_household_total_all_households
Tenure of household: Owned,tenure_of_household_owned
Tenure of household: Owned: Owns outright,tenure_of_household_owned_owns_outright
Tenure of household: Owned: Owns with a mortgage or loan,tenure_of_household_owned_owns_with_a_mortgage_or_loan
Tenure of household: Shared ownership,tenure_of_household_shared_ownership
Tenure of household: Shared ownership: Shared ownership,tenure_of_household_shared_ownership_shared_ownership
Tenure of household: Social rented,tenure_of_household_social_rented
Tenure of household: Social rented: Rents from council or Local Authority,tenure_of_household_social_rented_rents_from_council_or_local_authority
Tenure of household: Social rented: Other social rented,tenure_of_household_social_rented_other_social_rented
Tenure of household: Private rented,tenure_of_household_private_rented



[df_tenure_2021_final] shape after renaming: (348, 16)



date,geography,geography code,tenure_of_household_total_all_households,tenure_of_household_owned,tenure_of_household_owned_owns_outright,tenure_of_household_owned_owns_with_a_mortgage_or_loan,tenure_of_household_shared_ownership,tenure_of_household_shared_ownership_shared_ownership,tenure_of_household_social_rented,tenure_of_household_social_rented_rents_from_council_or_local_authority,tenure_of_household_social_rented_other_social_rented,tenure_of_household_private_rented,tenure_of_household_private_rented_private_landlord_or_letting_agency,tenure_of_household_private_rented_other_private_rented,tenure_of_household_lives_rent_free
2021,Hartlepool,E06000001,40929,23834,12761,11073,239,239,9594,3986,5608,7194,6338,856,68
2021,Middlesbrough,E06000002,60265,33137,17224,15913,330,330,14006,6773,7233,12677,11286,1391,115
2021,Redcar and Cleveland,E06000003,61638,39854,22649,17205,346,346,11649,5269,6380,9748,8410,1338,41
2021,Stockton-on-Tees,E06000004,83756,55412,27726,27686,385,385,13673,7204,6469,14207,12388,1819,79
2021,Darlington,E06000005,48915,30696,16222,14474,190,190,7931,5112,2819,10078,9035,1043,20



[df_tenure_2021_final] Total missing values: 0
[df_tenure_2021_final] No missing values found.

--------------------------------------------------
>>> Processing: Tenure (2011 Census)

Overview of Original Feature Names and Corresponding Simplified Identifiers:



Original Column Name,Renamed Column
Tenure: All categories: Tenure; measures: Value,all_categories_tenure
Tenure: Owned: Total; measures: Value,owned_total
Tenure: Owned: Owned outright; measures: Value,owned_owned_outright
Tenure: Owned: Owned with a mortgage or loan; measures: Value,owned_owned_with_a_mortgage_or_loan
Tenure: Shared ownership (part owned and part rented); measures: Value,shared_ownership_part_owned_and_part_rented
Tenure: Social rented: Total; measures: Value,social_rented_total
Tenure: Social rented: Rented from council (Local Authority); measures: Value,social_rented_rented_from_council_local_authority
Tenure: Social rented: Other social rented; measures: Value,social_rented_other_social_rented
Tenure: Private rented: Total; measures: Value,private_rented_total
Tenure: Private rented: Private landlord or letting agency; measures: Value,private_rented_private_landlord_or_letting_agency



[df_tenure_2011_final] shape after renaming: (348, 18)



date,geography,geography code,Rural Urban,all_categories_tenure,owned_total,owned_owned_outright,owned_owned_with_a_mortgage_or_loan,shared_ownership_part_owned_and_part_rented,social_rented_total,social_rented_rented_from_council_local_authority,social_rented_other_social_rented,private_rented_total,private_rented_private_landlord_or_letting_agency,private_rented_employer_of_a_household_member,private_rented_relative_or_friend_of_household_member,private_rented_other,living_rent_free
2011,Darlington,E06000005,Total,46670,30289,13810,16479,178,7228,5238,1990,8513,7758,112,546,97,462
2011,County Durham,E06000047,Total,223803,147347,69435,77912,488,45004,28559,16445,28142,25498,185,2121,338,2822
2011,Hartlepool,E06000001,Total,40434,24194,11090,13104,200,9515,4016,5499,5971,5525,26,349,71,554
2011,Middlesbrough,E06000002,Total,57203,32723,15200,17523,336,13654,6049,7605,9509,8648,98,570,193,981
2011,Northumberland,E06000057,Total,138534,91207,46086,45121,510,25841,14820,11021,18417,16225,434,1388,370,2559



[df_tenure_2011_final] Total missing values: 0
[df_tenure_2011_final] No missing values found.


<a id="subsection-18-align-shared-features-for-accommodation-type-2011-and-2021"></a>
#### 1.8: Align Shared Features for Accommodation Type (2011 and 2021)

In this step, we aligned and stacked shared accommodation type features from the 2011 and 2021 census datasets into a unified DataFrame.

We defined a set of standardized 2021 target features and used token- and character-level similarity methods to map corresponding 2011 columns. After ensuring column alignment, we vertically concatenated the two datasets, inspected representative samples from both years, and checked for missing values across the combined dataset.

In [9]:
# Step 1: Define a list of ID columns to retain during merging
_ID_COLS: List[str] = ["date", "geography", "geography code"]

# Step 2: Define the target features for 2021 accommodation type
_TARGET_21: List[str] = [
    "accommodation_type_detached",
    "accommodation_type_semi_detached",
    "accommodation_type_terraced",
    "accommodation_type_in_a_purpose_built_block_of_flats_or_tenement",
    "accommodation_type_part_of_a_converted_or_shared_house_including_bedsits",
    "accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop",
    "accommodation_type_a_caravan_or_other_mobile_or_temporary_structure",
]

# Step 3: Define stopwords to ignore during token matching
_STOP: Set[str] = {
    "a", "an", "the", "of", "in", "on", "and", "or", "other",
    "accommodation", "type", "dwelling", "unshared", "shared",
    "whole", "house", "bungalow", "flat", "maisonette", "apartment",
    "total", "all", "including", "end", "terrace", "purpose", "built",
    "block", "flats", "tenement", "part", "former"
}

# Step 4: Compile a regex for basic tokenization
_tok_rx = re.compile(r"[^a-z ]+")

# Step 5: Tokenize and lowercase a column name, removing stopwords
def _tokens(label: str) -> Set[str]:
    return {w for w in _tok_rx.sub(" ", label.lower()).split() if w and w not in _STOP}

# Step 6: Compute Jaccard similarity between two token sets
def _jaccard(t1: Set[str], t2: Set[str]) -> float:
    return 0.0 if not t1 or not t2 else len(t1 & t2) / len(t1 | t2)

# Step 7: Compute character-level similarity using SequenceMatcher
def _char_sim(a: str, b: str) -> float:
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Step 8: Find the best matching 2011 feature for a given 2021 feature
def _best_2011_for(
    col_21: str,
    pool_11: List[str],
    used_11: Set[str],
    thr_tok: float = 0.25,
    thr_char: float = 0.30,
) -> Optional[str]:
    tgt_tok = _tokens(col_21)
    best, best_score = None, 0.0
    for col_11 in pool_11:
        if col_11 in used_11:
            continue
        tok_11 = _tokens(col_11)
        score = _jaccard(tok_11, tgt_tok)
        if score < thr_tok:
            score = max(score, _char_sim(col_11, col_21))
        if score > best_score:
            best, best_score = col_11, score
    return best if best_score >= thr_tok or best_score >= thr_char else None

# Step 9: Build a mapping from 2011 feature names to 2021 target features
def _build_mapping(feat_11: List[str]) -> Dict[str, str]:
    mapping, used = {}, set()
    for tgt in _TARGET_21:
        src = _best_2011_for(tgt, feat_11, used)
        if src:
            mapping[src] = tgt
            used.add(src)
    return mapping

# Step 10: Align and vertically stack shared accommodation type features
def stack_accommodation_2011_2021(df_2011: pd.DataFrame, df_2021: pd.DataFrame) -> pd.DataFrame:

    feat_11 = [c for c in df_2011.columns if c not in _ID_COLS]
    mapping = _build_mapping(feat_11)

    df11_sel = (
        df_2011
        .rename(columns=mapping)
        .loc[:, _ID_COLS + _TARGET_21]
    )

    missing = [c for c in _TARGET_21 if c not in df11_sel.columns]
    if missing:
        raise ValueError(f"Could not map these target columns from 2011 data: {missing}")

    df21_sel = df_2021.loc[:, _ID_COLS + _TARGET_21]

    assert list(df11_sel.columns) == list(df21_sel.columns), "Column order mismatch"

    df_combined = pd.concat([df11_sel, df21_sel], ignore_index=True)

    print(f"\n[df_combined_accommodation] shape after alignment: {df_combined.shape}\n")

    df11_preview = df_combined[df_combined["date"] == 2011].head()
    df21_preview = df_combined[df_combined["date"] == 2021].head()
    ellipsis_row = pd.DataFrame([["..."] * df_combined.shape[1]], columns=df_combined.columns)

    df_preview = pd.concat([
        df11_preview,
        ellipsis_row,
        df21_preview,
        ellipsis_row,
    ], ignore_index=True)

    display(df_preview.style.hide(axis="index"))

    total_missing = df_combined.isnull().sum().sum()
    if total_missing > 0:
        print(f"\n[df_combined_accommodation] Total missing values: {total_missing}")
        missing_by_column = df_combined.isnull().mean().round(4) * 100
        print("[df_combined_accommodation] Missing value percentage by column:")
        print(missing_by_column[missing_by_column > 0].round(2).astype(str) + '%')
    else:
        print(f"\n[df_combined_accommodation] Total missing values: 0")
        print("[df_combined_accommodation] No missing values found.")

    return df_combined

# Step 11: Run the stacking function on aligned 2011 and 2021 datasets
print("\n>>> Align accommodation type features from the 2011 and 2021 census datasets")
df_combined_accommodation = stack_accommodation_2011_2021(
    df_accommodation_2011_final,
    df_accommodation_2021_final,
)


>>> Align accommodation type features from the 2011 and 2021 census datasets

[df_combined_accommodation] shape after alignment: (696, 10)



date,geography,geography code,accommodation_type_detached,accommodation_type_semi_detached,accommodation_type_terraced,accommodation_type_in_a_purpose_built_block_of_flats_or_tenement,accommodation_type_part_of_a_converted_or_shared_house_including_bedsits,accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop,accommodation_type_a_caravan_or_other_mobile_or_temporary_structure
2011,Darlington,E06000005,8209,17835,14379,5053,46670,6099,30
2011,County Durham,E06000047,43114,83225,84752,9941,223803,12436,228
2011,Hartlepool,E06000001,6822,13448,14933,4489,40434,5121,88
2011,Middlesbrough,E06000002,8001,23394,18329,6389,57203,7325,21
2011,Northumberland,E06000057,35120,50127,38345,11602,138534,14676,232
...,...,...,...,...,...,...,...,...,...
2021,Hartlepool,E06000001,8086,14537,13469,3995,314,260,115
2021,Middlesbrough,E06000002,10389,25485,16768,6457,647,308,51
2021,Redcar and Cleveland,E06000003,12449,27942,15666,4005,791,388,84
2021,Stockton-on-Tees,E06000004,22209,35494,18298,6444,723,334,79



[df_combined_accommodation] Total missing values: 0
[df_combined_accommodation] No missing values found.


<a id="subsection-19-align-shared-features-for-tenure-2011-and-2021"></a>
#### 1.9: Align Shared Features for Tenure (2011 and 2021)

In this step, we repeated the alignment and vertical stacking procedure for the tenure datasets from 2011 and 2021.

We matched feature columns using similarity scoring, renamed 2011 features to match the 2021 schema, and concatenated the datasets into a single DataFrame. We displayed samples from both years, confirmed column consistency, and reported missing value statistics for the stacked dataset.

In [10]:
# Step 1: Define ID and target feature columns for tenure data
_ID_COLS: List[str] = ["date", "geography", "geography code"]

_TARGET_21: List[str] = [
    "tenure_of_household_owned_owns_outright",
    "tenure_of_household_owned_owns_with_a_mortgage_or_loan",
    "tenure_of_household_shared_ownership",
    "tenure_of_household_social_rented",
    "tenure_of_household_social_rented_rents_from_council_or_local_authority",
    "tenure_of_household_social_rented_other_social_rented",
    "tenure_of_household_private_rented_private_landlord_or_letting_agency",
    "tenure_of_household_lives_rent_free"
]

# Step 2: Define stopwords and tokenizer for matching
_STOP: Set[str] = {"a", "an", "the", "of", "in", "on", "and", "or"}
_tok_rx = re.compile(r"[^a-z ]+")

# Step 3: Helper functions for string matching
def _tokens(label: str) -> Set[str]:
    return {w for w in _tok_rx.sub(" ", label.lower()).split() if w not in _STOP}

def _jaccard(t1: Set[str], t2: Set[str]) -> float:
    return 0.0 if not t1 or not t2 else len(t1 & t2) / len(t1 | t2)

def _char_sim(a: str, b: str) -> float:
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def _best_2011_for(
    col_21: str,
    pool_11: List[str],
    used_11: Set[str],
    thr_tok: float = 0.25,
    thr_char: float = 0.30,
) -> Optional[str]:
    tgt_tok = _tokens(col_21)
    best, best_score = None, 0.0
    for col_11 in pool_11:
        if col_11 in used_11:
            continue
        tok_11 = _tokens(col_11)
        score = _jaccard(tok_11, tgt_tok)
        if score < thr_tok:
            score = max(score, _char_sim(col_11, col_21))
        if score > best_score:
            best, best_score = col_11, score
    return best if best_score >= thr_tok or best_score >= thr_char else None

def _build_mapping(feat_11: List[str]) -> Dict[str, str]:
    mapping, used = {}, set()
    for tgt in _TARGET_21:
        src = _best_2011_for(tgt, feat_11, used)
        if src:
            mapping[src] = tgt
            used.add(src)
    return mapping

# Step 4: Main function to align and stack tenure features
def stack_tenure_2011_2021(df_2011: pd.DataFrame, df_2021: pd.DataFrame) -> pd.DataFrame:

    feat_11 = [c for c in df_2011.columns if c not in _ID_COLS]
    mapping = _build_mapping(feat_11)

    df11_sel = (
        df_2011
        .rename(columns=mapping)
        .loc[:, _ID_COLS + _TARGET_21]
    )

    missing = [c for c in _TARGET_21 if c not in df11_sel.columns]
    if missing:
        raise ValueError(f"Could not map these target columns from 2011 data: {missing}")

    df21_sel = df_2021.loc[:, _ID_COLS + _TARGET_21]

    assert list(df11_sel.columns) == list(df21_sel.columns), "Column order mismatch"

    df_combined = pd.concat([df11_sel, df21_sel], ignore_index=True)

    print(f"\n[df_combined_tenure] shape after alignment: {df_combined.shape}\n")

    df11_preview = df_combined[df_combined["date"] == 2011].head()
    df21_preview = df_combined[df_combined["date"] == 2021].head()
    ellipsis_row = pd.DataFrame([["..."] * df_combined.shape[1]], columns=df_combined.columns)

    df_preview = pd.concat([
        df11_preview,
        ellipsis_row,
        df21_preview,
        ellipsis_row,
    ], ignore_index=True)

    display(df_preview.style.hide(axis="index"))

    total_missing = df_combined.isnull().sum().sum()
    if total_missing > 0:
        print(f"\n[df_combined_tenure] Total missing values: {total_missing}")
        missing_by_column = df_combined.isnull().mean().round(4) * 100
        print("[df_combined_tenure] Missing value percentage by column:")
        print(missing_by_column[missing_by_column > 0].round(2).astype(str) + '%')
    else:
        print(f"\n[df_combined_tenure] Total missing values: 0")
        print("[df_combined_tenure] No missing values found.")

    return df_combined

# Step 5: Run the tenure stacking on aligned 2011 and 2021 datasets
print("\n>>> Align tenure features from the 2011 and 2021 census datasets")
df_combined_tenure = stack_tenure_2011_2021(
    df_tenure_2011_final,
    df_tenure_2021_final
)


>>> Align tenure features from the 2011 and 2021 census datasets

[df_combined_tenure] shape after alignment: (696, 11)



date,geography,geography code,tenure_of_household_owned_owns_outright,tenure_of_household_owned_owns_with_a_mortgage_or_loan,tenure_of_household_shared_ownership,tenure_of_household_social_rented,tenure_of_household_social_rented_rents_from_council_or_local_authority,tenure_of_household_social_rented_other_social_rented,tenure_of_household_private_rented_private_landlord_or_letting_agency,tenure_of_household_lives_rent_free
2011,Darlington,E06000005,112,16479,546,7228,5238,1990,7758,97
2011,County Durham,E06000047,185,77912,2121,45004,28559,16445,25498,338
2011,Hartlepool,E06000001,26,13104,349,9515,4016,5499,5525,71
2011,Middlesbrough,E06000002,98,17523,570,13654,6049,7605,8648,193
2011,Northumberland,E06000057,434,45121,1388,25841,14820,11021,16225,370
...,...,...,...,...,...,...,...,...,...,...
2021,Hartlepool,E06000001,12761,11073,239,9594,3986,5608,6338,68
2021,Middlesbrough,E06000002,17224,15913,330,14006,6773,7233,11286,115
2021,Redcar and Cleveland,E06000003,22649,17205,346,11649,5269,6380,8410,41
2021,Stockton-on-Tees,E06000004,27726,27686,385,13673,7204,6469,12388,79



[df_combined_tenure] Total missing values: 0
[df_combined_tenure] No missing values found.


<a id="section-2-bayesian-confirmatory-factor-analysis-of-housing-related-latent-constructs"></a>
### Part 2: Bayesian Confirmatory Factor Analysis of Housing-Related Latent Constructs

In this part, we constructed interpretable latent indices from census microdata using Bayesian confirmatory factor analysis (CFA), enabling dimensionality reduction and index-based comparison across space and time.

We began by applying Bayesian CFA to the 2011 census data to estimate four latent constructs—deprivation, housing strain, tenure instability, and mobility access—based on predefined indicator groups (2.1). These latent scores and their uncertainty intervals were merged back into the dataset for reporting. We then reshaped the wide-format latent outputs into long-format structures suitable for Tableau visualization, with each row representing a latent–indicator pair (2.2). Next, we applied a similar modeling pipeline to the vertically stacked accommodation data from 2011 and 2021, generating two latent constructs—accommodation density and temporary dwelling (2.3)—and then pivoted them into a year-wide format for Tableau visualization (2.4). We repeated this CFA-based modeling and reshaping process for tenure-related indicators across 2011 and 2021, where we estimated home ownership stability and rental insecurity scores (2.5), and prepared wide-format outputs for Tableau visualization (2.6).

<a id="subsection-21-estimate-latent-socio-economic-constructs-using-bayesian-confirmatory-factor-analysis"></a>
#### 2.1: Estimate Latent Socio-Economic Constructs using Bayesian Confirmatory Factor Analysis

In this step, we applied Bayesian confirmatory factor analysis (CFA) to the cleaned and aligned 2011 census dataset in order to estimate four latent constructs based on predefined indicator groupings.

We defined indicator–latent variable mappings, standardized the indicators, and fitted a Bayesian CFA model using PyMC and automatic variational inference. Posterior means and 95% credible intervals were extracted for each latent construct and merged into the original dataset. 

In [11]:
# Suppress convergence messages during variational inference for clean output
warnings.filterwarnings("ignore", category=UserWarning)
for name in ["pytensor", "pytensor.configdefaults", "pymc"]:
    logging.getLogger(name).setLevel(logging.ERROR)

class SilentConvergence(pm.callbacks.CheckParametersConvergence):
    def _print_message(self, *a, **k):
        pass

# Step 1: Define a function to run Bayesian confirmatory factor analysis
def run_bayesian_cfa(df_final: pd.DataFrame) -> pd.DataFrame:

    # Define latent constructs and their observed indicators
    indicator_map = {
        "latent_deprivation_index": [
            "social_rented_total",
            "social_rented_rented_from_council_local_authority",
            "occupancy_rating_bedrooms_of_0",
        ],
        
        "housing_strain_score": [
            "unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits",
            "occupancy_rating_bedrooms_of_minus_1",
            "unshared_dwelling_caravan_or_other_mobile_or_temporary_structure",
        ],
        
        "tenure_instability_score": [
            "private_rented_total",
            "shared_ownership_part_owned_and_part_rented",
            "private_rented_relative_or_friend_of_household_member",
        ],
        
        "low_mobility_access_index": [
            "no_cars_or_vans_in_household",
            "1_car_or_van_in_household",
            "sum_of_all_cars_or_vans_in_the_area",
        ],
    }

    
    # Extract list of latent variable names
    latent_names = list(indicator_map.keys())

    # Collect all unique indicator columns
    chosen_cols = []
    for latent in latent_names:
        for col in indicator_map[latent]:
            if col not in chosen_cols:
                chosen_cols.append(col)

    # Create a dataframe with selected indicators
    df_model = df_final[chosen_cols].copy()
    if df_model.columns.duplicated().any():
        df_model = df_model.loc[:, ~df_model.columns.duplicated()].copy()

    # Standardize indicators (z-score normalization)
    means = df_model.mean()
    stds = df_model.std().replace(0, 1e-6)
    df_model_std = (df_model - means) / stds

    # Convert dataframe to numpy array
    Y = df_model_std.values
    n_obs, n_ind = Y.shape
    n_latent = len(latent_names)

    # Build a vector mapping indicators to their latent variables
    ind2latent_idx = np.array([
        next(
            lat_idx
            for lat_idx, lat in enumerate(latent_names)
            if col in indicator_map[lat]
        )
        for col in df_model.columns
    ])

    # Specify model coordinate names
    coords = {
        "obs": range(n_obs),
        "indicator": df_model.columns,
        "latent": latent_names,
    }

    # Step 2: Define the Bayesian CFA model
    with pm.Model(coords=coords) as cfa_model:

        # Latent factors for each observation
        F = pm.Normal("F", 0.0, 1.0, dims=("obs", "latent"))

        # Intercepts for each indicator
        alpha = pm.Normal("alpha", 0.0, 1.0, dims="indicator")

        # Raw loadings (constrained to be non-negative)
        lambda_raw = pm.HalfNormal("lambda_raw", 1.0, dims="indicator")

        # Construct a sparse loading matrix that maps indicators to latent variables
        Lambda = pm.Deterministic(
            "Lambda",
            pm.math.stack(
                [pm.math.switch(ind2latent_idx == k, lambda_raw, 0.0)
                 for k in range(n_latent)]
            ).T,
        )

        # Residual standard deviation for each indicator
        sigma = pm.HalfNormal("sigma", 1.0, dims="indicator")

        # Predicted means for indicators
        mu = alpha + pm.math.dot(F, Lambda.T)

        # Observed data likelihood
        pm.Normal("Y_obs", mu, sigma, observed=Y, dims=("obs", "indicator"))

        # Fit the model using automatic variational inference
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
            approx = pm.fit(
                method="advi",
                n=6_000,
                callbacks=[SilentConvergence(tolerance=1e-4)],
                progressbar=False,
                random_seed=42,
            )

            # Sample from the approximated posterior
            trace = approx.sample(draws=2_000, random_seed=42)

    # Collapse posterior samples across chains
    posterior = trace.posterior.stack(sample=("chain", "draw"))

    # Step 3: Extract posterior summaries for each latent variable
    for lat in latent_names:
        da = posterior["F"].sel(latent=lat)
        df_model[f"{lat}_mean"] = da.mean(dim="sample").values
        df_model[f"{lat}_hdi_2_5"] = np.percentile(da.values, 2.5, axis=1)
        df_model[f"{lat}_hdi_97_5"] = np.percentile(da.values, 97.5, axis=1)

    # Step 4: Merge posterior estimates back into the original dataset
    df_enriched = df_final.merge(
        df_model[[c for c in df_model.columns if "_mean" in c or "_hdi" in c]],
        left_index=True,
        right_index=True,
        how="left",
    )

    return df_enriched

# Step 5: Run Bayesian CFA on the cleaned 2011 dataset
df_combined_2011_with_latents = run_bayesian_cfa(df_combined_2011_final)

# Step 6: Select base and indicator columns for reporting
base_cols = ["date", "geography", "geography code", "Rural Urban"]
indicator_map = {
    "latent_deprivation_index": [
        "social_rented_total",
        "social_rented_rented_from_council_local_authority",
        "occupancy_rating_bedrooms_of_0",
    ],
    "housing_strain_score": [
        "unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits",
        "occupancy_rating_bedrooms_of_minus_1",
        "unshared_dwelling_caravan_or_other_mobile_or_temporary_structure",
    ],
    "tenure_instability_score": [
        "private_rented_total",
        "shared_ownership_part_owned_and_part_rented",
        "private_rented_relative_or_friend_of_household_member",
    ],
    "low_mobility_access_index": [
        "no_cars_or_vans_in_household",
        "1_car_or_van_in_household",
        "sum_of_all_cars_or_vans_in_the_area",
    ],
}


# Step 7: Build the final list of selected columns
all_selected_cols = base_cols.copy()
for latent, indicators in indicator_map.items():
    all_selected_cols.extend(indicators)
    all_selected_cols.append(f"{latent}_mean")
    all_selected_cols.append(f"{latent}_hdi_2_5")
    all_selected_cols.append(f"{latent}_hdi_97_5")
all_selected_cols = list(dict.fromkeys(all_selected_cols))

# Step 8: Filter and sort the dataset for output
df_2011_with_latents = df_combined_2011_with_latents[all_selected_cols].copy()
df_2011_with_latents = df_2011_with_latents.sort_values(by="geography code")

# Step 9: Rename latent variable columns for clarity
rename_dict = {
    "latent_deprivation_index_mean": "latent_deprivation_index",
    "latent_deprivation_index_hdi_2_5": "latent_deprivation_index_lower_2_5",
    "latent_deprivation_index_hdi_97_5": "latent_deprivation_index_upper_97_5",
    "housing_strain_score_mean": "housing_strain_score",
    "housing_strain_score_hdi_2_5": "housing_strain_score_lower_2_5",
    "housing_strain_score_hdi_97_5": "housing_strain_score_upper_97_5",
    "tenure_instability_score_mean": "tenure_instability_score",
    "tenure_instability_score_hdi_2_5": "tenure_instability_score_lower_2_5",
    "tenure_instability_score_hdi_97_5": "tenure_instability_score_upper_97_5",
    "low_mobility_access_index_mean": "low_mobility_access_index",
    "low_mobility_access_index_hdi_2_5": "low_mobility_access_index_lower_2_5",
    "low_mobility_access_index_hdi_97_5": "low_mobility_access_index_upper_97_5"
}

df_2011_with_latents = df_2011_with_latents.rename(columns=rename_dict)

# Step 10: Display the final latent-enriched dataset and report missingness
print(f"\n[df_2011_with_latents] shape: {df_2011_with_latents.shape}\n")
display(
    df_2011_with_latents.head()
    .style
    .hide(axis="index")
    .set_properties(**{"text-align": "left"})
    .set_table_styles([{
        "selector": "th",
        "props": [("text-align", "left")]
    }])
)


total_missing = df_2011_with_latents.isna().sum().sum()
if total_missing:
    print(f"\n[df_2011_with_latents] Total missing values: {total_missing}")
    missing_pct = (
        df_2011_with_latents.isna().mean().mul(100).round(2)
    )
    print(
        "[df_2011_with_latents] Missing % by column (non-zero only):\n",
        missing_pct[missing_pct > 0].astype(str) + " %"
    )
else:
    print(f"\n[df_2011_with_latents] Total missing values: 0")
    print("[df_2011_with_latents] No missing values found.")


[df_2011_with_latents] shape: (348, 28)



date,geography,geography code,Rural Urban,social_rented_total,social_rented_rented_from_council_local_authority,occupancy_rating_bedrooms_of_0,latent_deprivation_index,latent_deprivation_index_lower_2_5,latent_deprivation_index_upper_97_5,unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits,occupancy_rating_bedrooms_of_minus_1,unshared_dwelling_caravan_or_other_mobile_or_temporary_structure,housing_strain_score,housing_strain_score_lower_2_5,housing_strain_score_upper_97_5,private_rented_total,shared_ownership_part_owned_and_part_rented,private_rented_relative_or_friend_of_household_member,tenure_instability_score,tenure_instability_score_lower_2_5,tenure_instability_score_upper_97_5,no_cars_or_vans_in_household,1_car_or_van_in_household,sum_of_all_cars_or_vans_in_the_area,low_mobility_access_index,low_mobility_access_index_lower_2_5,low_mobility_access_index_upper_97_5
2011,Hartlepool,E06000001,Total,9515,4016,9576,-0.245244,-1.672152,1.105510,330,947,88,-0.328652,-1.725572,1.035248,5971,200,349,-0.395896,-1.801641,0.981660,14268,16573,38269,-0.361517,-1.745415,0.997908
2011,Middlesbrough,E06000002,Total,13654,6049,15031,-0.011676,-1.406593,1.335286,589,2246,21,-0.294337,-1.746948,1.138367,9509,336,570,-0.136005,-1.540073,1.216073,21488,22963,51821,-0.171275,-1.458866,1.166301
2011,Redcar and Cleveland,E06000003,Total,11568,5999,13404,-0.107442,-1.449018,1.331043,911,1280,25,-0.298028,-1.790799,1.181501,7434,256,578,-0.230512,-1.635224,1.101004,16935,25367,64747,-0.119847,-1.470638,1.244253
2011,Stockton-on-Tees,E06000004,Total,13625,7285,16962,0.054700,-1.282923,1.402989,716,2118,63,-0.227944,-1.660591,1.210238,10387,255,788,-0.034779,-1.407901,1.363594,20524,32728,91623,0.137308,-1.240784,1.624186
2011,Darlington,E06000005,Total,7228,5238,11450,-0.227373,-1.520811,1.096209,778,1149,30,-0.291799,-1.814767,1.203729,8513,178,546,-0.245492,-1.641622,1.156184,13052,20682,49794,-0.296203,-1.781142,1.106872



[df_2011_with_latents] Total missing values: 0
[df_2011_with_latents] No missing values found.


<a id="subsection-22-reshape-latent-estimates-output-dataframe-into-long-format-for-tableau-visualization"></a>
#### 2.2: Reshape Latent Estimates Output Dataframe into Long Format for Tableau Visualization

In this step, we restructured the latent socio-economic index estimates from wide format into long format to support Tableau visualization. For each geography, we looped over the predefined latent–indicator mapping and expanded each latent dimension into multiple rows corresponding to its observed features. Each row included the latent value, its 95% interval, and the associated feature value. The resulting long-format DataFrame was checked for missing values, previewed, and exported as a CSV file for Tableau visualization.

In [12]:
# Step 1: Define base metadata columns to retain in the long-format table
base_cols = ["date", "geography", "geography code", "Rural Urban"]

# Step 2: Define the latent index map and their associated observed indicators
indicator_map = {
    "latent_deprivation_index": [
        "social_rented_total",
        "social_rented_rented_from_council_local_authority",
        "occupancy_rating_bedrooms_of_0",
    ],
    "housing_strain_score": [
        "unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits",
        "occupancy_rating_bedrooms_of_minus_1",
        "unshared_dwelling_caravan_or_other_mobile_or_temporary_structure",
    ],
    "tenure_instability_score": [
        "private_rented_total",
        "shared_ownership_part_owned_and_part_rented",
        "private_rented_relative_or_friend_of_household_member",
    ],
    "low_mobility_access_index": [
        "no_cars_or_vans_in_household",
        "1_car_or_van_in_household",
        "sum_of_all_cars_or_vans_in_the_area",
    ],
}

# Step 3: Iterate over each row and reshape wide-format latent structure to long-format
rows = []
for _, row in df_2011_with_latents.iterrows():
    for latent, features in indicator_map.items():
        for feat in features:
            rows.append({
                "date": row["date"],
                "geography": row["geography"],
                "geography code": row["geography code"],
                "Rural Urban": row["Rural Urban"],
                "latent_deprivation_index_lower_2_5": row["latent_deprivation_index_lower_2_5"],
                "latent_deprivation_index_upper_97_5": row["latent_deprivation_index_upper_97_5"],
                "housing_strain_score_lower_2_5": row["housing_strain_score_lower_2_5"],
                "housing_strain_score_upper_97_5": row["housing_strain_score_upper_97_5"],
                "tenure_instability_score_lower_2_5": row["tenure_instability_score_lower_2_5"],
                "tenure_instability_score_upper_97_5": row["tenure_instability_score_upper_97_5"],
                "low_mobility_access_index_lower_2_5": row["low_mobility_access_index_lower_2_5"],
                "low_mobility_access_index_upper_97_5": row["low_mobility_access_index_upper_97_5"],
                "latent_index": latent,
                "index_value": row[latent],
                "feature_type": feat,
                "feature_value": row[feat]
            })

# Step 4: Create a new DataFrame with long-format latent-index rows
df_2011_with_latents_long_format = pd.DataFrame(rows)

# Step 5: Display the shape and preview of the reshaped data
print(f"\n[df_2011_with_latents_long_format] shape: {df_2011_with_latents_long_format.shape}\n")
display(
    df_2011_with_latents_long_format.head()
    .style
    .hide(axis="index")
    .set_properties(**{"text-align": "left"})
    .set_table_styles([{
        "selector": "th",
        "props": [("text-align", "left")]
    }])
)


# Step 6: Check for and report missing values in the reshaped dataset
total_missing = df_2011_with_latents_long_format.isna().sum().sum()
if total_missing:
    print(f"\n[df_2011_with_latents_long_format] Total missing values: {total_missing}")
    missing_pct = (
        df_2011_with_latents_long_format.isna().mean().mul(100).round(2)
    )
    print(
        "[df_2011_with_latents_long_format] Missing % by column (non-zero only):\n",
        missing_pct[missing_pct > 0].astype(str) + " %"
    )
else:
    print(f"\n[df_2011_with_latents_long_format] Total missing values: 0")
    print("[df_2011_with_latents_long_format] No missing values found.")

# Step 7: Prepare export path and directory for Tableau Visualization
for dashboard in ["Dashboard1", "Dashboard2"]:
    os.makedirs(dashboard, exist_ok=True)
    output_path = os.path.join(dashboard, "England_Wales_SocioEconomic_Housing_Latent_Long_Format_2011.csv")
    if os.path.exists(output_path):
        print(f"\n[SKIPPED] File already exists: {output_path}")
    else:
        df_2011_with_latents_long_format.to_csv(output_path, index=False)
        print(f"\n[SAVED] The DataFrame has been exported to: {output_path} (Ready for Tableau Visualization).")


[df_2011_with_latents_long_format] shape: (4176, 16)



date,geography,geography code,Rural Urban,latent_deprivation_index_lower_2_5,latent_deprivation_index_upper_97_5,housing_strain_score_lower_2_5,housing_strain_score_upper_97_5,tenure_instability_score_lower_2_5,tenure_instability_score_upper_97_5,low_mobility_access_index_lower_2_5,low_mobility_access_index_upper_97_5,latent_index,index_value,feature_type,feature_value
2011,Hartlepool,E06000001,Total,-1.672152,1.105510,-1.725572,1.035248,-1.801641,0.981660,-1.745415,0.997908,latent_deprivation_index,-0.245244,social_rented_total,9515
2011,Hartlepool,E06000001,Total,-1.672152,1.105510,-1.725572,1.035248,-1.801641,0.981660,-1.745415,0.997908,latent_deprivation_index,-0.245244,social_rented_rented_from_council_local_authority,4016
2011,Hartlepool,E06000001,Total,-1.672152,1.105510,-1.725572,1.035248,-1.801641,0.981660,-1.745415,0.997908,latent_deprivation_index,-0.245244,occupancy_rating_bedrooms_of_0,9576
2011,Hartlepool,E06000001,Total,-1.672152,1.105510,-1.725572,1.035248,-1.801641,0.981660,-1.745415,0.997908,housing_strain_score,-0.328652,unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits,330
2011,Hartlepool,E06000001,Total,-1.672152,1.105510,-1.725572,1.035248,-1.801641,0.981660,-1.745415,0.997908,housing_strain_score,-0.328652,occupancy_rating_bedrooms_of_minus_1,947



[df_2011_with_latents_long_format] Total missing values: 0
[df_2011_with_latents_long_format] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard1\England_Wales_SocioEconomic_Housing_Latent_Long_Format_2011.csv (Ready for Tableau Visualization).

[SAVED] The DataFrame has been exported to: Dashboard2\England_Wales_SocioEconomic_Housing_Latent_Long_Format_2011.csv (Ready for Tableau Visualization).


<a id="subsection-23-bayesian-cfa-of-accommodation-structure-2011-vs-2021-modeling"></a>
#### 2.3: Bayesian CFA of Accommodation Structure (2011 vs 2021 Modeling)

In this step, we defined a mapping of observed indicators to two latent constructs related to housing—accommodation density and temporary dwelling—and estimated them using Bayesian confirmatory factor analysis (CFA) across combined data from 2011 and 2021.

We standardized indicators, defined a PyMC probabilistic model, and performed automatic variational inference to compute posterior means and credible intervals for each latent dimension. The enriched dataset was merged, year-tagged, previewed, and exported for Tableau visualization.

In [13]:
# Suppress convergence messages during variational inference for clean output
warnings.filterwarnings("ignore", category=UserWarning)
for name in ["pytensor", "pytensor.configdefaults", "pymc"]:
    logging.getLogger(name).setLevel(logging.ERROR)

# Step 1: Define indicator-to-latent mappings for accommodation-related constructs
indicator_map = {
    "accommodation_density_score": [
        "accommodation_type_terraced",
        "accommodation_type_part_of_a_converted_or_shared_house_including_bedsits",
        "accommodation_type_in_a_purpose_built_block_of_flats_or_tenement",
    ],
    "temporary_dwelling_index": [
        "accommodation_type_a_caravan_or_other_mobile_or_temporary_structure",
        "accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop",
    ],
}

# Step 2: Define a function to perform Bayesian CFA using the indicator mapping
def run_bayesian_cfa(df_final: pd.DataFrame, indicator_map: dict) -> pd.DataFrame:
    latent_names = list(indicator_map.keys())

    # Select all unique indicator columns from the mapping
    chosen_cols = []
    for latent in latent_names:
        for col in indicator_map[latent]:
            if col not in chosen_cols:
                chosen_cols.append(col)

    # Subset and standardize the selected indicator data
    df_model = df_final[chosen_cols].copy()
    if df_model.columns.duplicated().any():
        df_model = df_model.loc[:, ~df_model.columns.duplicated()].copy()

    means = df_model.mean()
    stds = df_model.std().replace(0, 1e-6)
    df_model_std = (df_model - means) / stds

    Y = df_model_std.values
    n_obs, n_ind = Y.shape
    n_latent = len(latent_names)

    # Create a vector mapping indicators to their respective latent index
    ind2latent_idx = np.array([
        next(
            lat_idx
            for lat_idx, lat in enumerate(latent_names)
            if col in indicator_map[lat]
        )
        for col in df_model.columns
    ])

    coords = {
        "obs": range(n_obs),
        "indicator": df_model.columns,
        "latent": latent_names,
    }

    # Build the PyMC model for CFA and fit it with ADVI
    with pm.Model(coords=coords) as cfa_model:
        F = pm.Normal("F", 0.0, 1.0, dims=("obs", "latent"))
        alpha = pm.Normal("alpha", 0.0, 1.0, dims="indicator")
        lambda_raw = pm.HalfNormal("lambda_raw", 1.0, dims="indicator")

        Lambda = pm.Deterministic(
            "Lambda",
            pm.math.stack(
                [pm.math.switch(ind2latent_idx == k, lambda_raw, 0.0)
                 for k in range(n_latent)]
            ).T,
        )

        sigma = pm.HalfNormal("sigma", 1.0, dims="indicator")
        mu = alpha + pm.math.dot(F, Lambda.T)

        pm.Normal("Y_obs", mu, sigma, observed=Y, dims=("obs", "indicator"))
        
        with contextlib.redirect_stdout(io.StringIO()), \
             contextlib.redirect_stderr(io.StringIO()):
            approx = pm.fit(
                method="advi",
                n=6_000, 
                callbacks=[SilentConvergence(tolerance=1e-4)],
                progressbar=False,
                random_seed=42,
            )
            trace = approx.sample(draws=2_000, random_seed=42)     

    posterior = trace.posterior.stack(sample=("chain", "draw"))

    # Extract latent variable summaries and append to the dataset
    for lat in latent_names:
        da = posterior["F"].sel(latent=lat)
        df_model[f"{lat}_mean"] = da.mean(dim="sample").values
        df_model[f"{lat}_hdi_2_5"] = np.percentile(da.values, 2.5, axis=1)
        df_model[f"{lat}_hdi_97_5"] = np.percentile(da.values, 97.5, axis=1)

    df_enriched = df_final.merge(
        df_model[[c for c in df_model.columns if "_mean" in c or "_hdi" in c]],
        left_index=True,
        right_index=True,
        how="left",
    )

    return df_enriched

# Step 3: Apply CFA to the combined accommodation dataset
df_with_accommodation_latents = run_bayesian_cfa(
    df_combined_accommodation, 
    indicator_map=indicator_map
)

# Step 4: Select output columns including indicators and latent scores
base_cols = ["date", "geography", "geography code"]
all_selected_cols = base_cols.copy()
for latent, indicators in indicator_map.items():
    all_selected_cols.extend(indicators)
    all_selected_cols.append(f"{latent}_mean")
    all_selected_cols.append(f"{latent}_hdi_2_5")
    all_selected_cols.append(f"{latent}_hdi_97_5")
all_selected_cols = list(dict.fromkeys(all_selected_cols))

# Step 5: Build and sort final output DataFrame
df_accommodation_latents_final = df_with_accommodation_latents[all_selected_cols].copy()
df_accommodation_latents_final = df_accommodation_latents_final.sort_values(by="geography code")

# Step 6: Rename columns for clearer interpretation
rename_dict = {
    "accommodation_density_score_mean": "accommodation_density_score",
    "accommodation_density_score_hdi_2_5": "accommodation_density_score_lower_2_5",
    "accommodation_density_score_hdi_97_5": "accommodation_density_score_upper_97_5",
    "temporary_dwelling_index_mean": "temporary_dwelling_index",
    "temporary_dwelling_index_hdi_2_5": "temporary_dwelling_index_lower_2_5",
    "temporary_dwelling_index_hdi_97_5": "temporary_dwelling_index_upper_97_5"
}
df_accommodation_latents_final = df_accommodation_latents_final.rename(columns=rename_dict)

print(f"\n[df_accommodation_latents_final] shape: {df_accommodation_latents_final.shape}\n")

# Step 7: Preview stacked results for 2011 and 2021
if not set(df_accommodation_latents_final["date"]).issuperset({2011, 2021}):
    print("[WARN] Cannot generate year-separated preview: 2011 or 2021 data missing.")
else:
    df11_preview = df_accommodation_latents_final[df_accommodation_latents_final["date"] == 2011].head()
    df21_preview = df_accommodation_latents_final[df_accommodation_latents_final["date"] == 2021].head()
    ellipsis_row = pd.DataFrame(
        [["..."] * df_accommodation_latents_final.shape[1]],
        columns=df_accommodation_latents_final.columns
    )

    df_preview = pd.concat([
        df11_preview,
        ellipsis_row,
        df21_preview,
        ellipsis_row,
    ], ignore_index=True)

    display(
        df_preview
        .style
        .hide(axis="index")
        .set_properties(**{"text-align": "left"})
        .set_table_styles([{
            "selector": "th",
            "props": [("text-align", "left")]
        }])
    )

# Step 8: Check for missing values in the final dataset
total_missing = df_accommodation_latents_final.isna().sum().sum()
if total_missing:
    print(f"\n[df_accommodation_latents_final] Total missing values: {total_missing}")
    missing_pct = (
        df_accommodation_latents_final.isna().mean().mul(100).round(2)
    )
    print(
        "[df_accommodation_latents_final] Missing % by column (non-zero only):\n",
        missing_pct[missing_pct > 0].astype(str) + " %"
    )
else:
    print(f"\n[df_accommodation_latents_final] Total missing values: 0")
    print("[df_accommodation_latents_final] No missing values found.")

# Step 9: Save output to CSV format for Tableau visualization
output_dir = "Dashboard3"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Accommodation_Type_2011_2021.csv")
if os.path.exists(output_path):
    print(f"\n[SKIPPED] File already exists: {output_path}")
else:
    df_accommodation_latents_final.to_csv(output_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_path} (Ready for Tableau Visualization).")


[df_accommodation_latents_final] shape: (696, 14)



date,geography,geography code,accommodation_type_terraced,accommodation_type_part_of_a_converted_or_shared_house_including_bedsits,accommodation_type_in_a_purpose_built_block_of_flats_or_tenement,accommodation_density_score,accommodation_density_score_lower_2_5,accommodation_density_score_upper_97_5,accommodation_type_a_caravan_or_other_mobile_or_temporary_structure,accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop,temporary_dwelling_index,temporary_dwelling_index_lower_2_5,temporary_dwelling_index_upper_97_5
2011,Hartlepool,E06000001,14933,40434,4489,-0.134924,-1.584055,1.229300,88,5121,-0.177368,-1.758532,1.437358
2011,Middlesbrough,E06000002,18329,57203,6389,0.027029,-1.429529,1.457549,21,7325,-0.173653,-1.745472,1.466239
2011,Redcar and Cleveland,E06000003,16582,59605,4107,-0.061437,-1.486261,1.371608,25,5472,-0.220456,-1.768346,1.344125
2011,Stockton-on-Tees,E06000004,18328,79159,7080,0.121014,-1.312728,1.563946,63,8228,-0.129247,-1.755100,1.404584
2011,Darlington,E06000005,14379,46670,5053,-0.125302,-1.610126,1.288531,30,6099,-0.203584,-1.787364,1.432224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Hartlepool,E06000001,13469,314,3995,-0.336822,-1.778539,1.085467,115,260,-0.204560,-1.773870,1.402039
2021,Middlesbrough,E06000002,16768,647,6457,-0.246363,-1.646341,1.128887,51,308,-0.315870,-2.005000,1.377593
2021,Redcar and Cleveland,E06000003,15666,791,4005,-0.324950,-1.712313,1.099004,84,388,-0.254159,-1.811620,1.357480
2021,Stockton-on-Tees,E06000004,18298,723,6444,-0.259466,-1.646561,1.190935,79,334,-0.226135,-1.797598,1.295423



[df_accommodation_latents_final] Total missing values: 0
[df_accommodation_latents_final] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard3\England_Wales_SocioEconomic_Accommodation_Type_2011_2021.csv (Ready for Tableau Visualization).


<a id="subsection-24-pivot-accommodation-latent-scores-into-year-wide-format-for-2011–2021-comparison"></a>
#### 2.4: Pivot Accommodation Latent Scores into Year-Wide Format for 2011–2021 Comparison

In this step, we transformed the accommodation latent scores from long format to wide format for year-wise comparison. Two latent indicators—accommodation density and temporary dwelling—were pivoted into 2011 and 2021 columns, indexed by geography and geography code. Column names were flattened and standardized, missing values were reported, and the resulting table was exported as a Tableau-ready CSV file.

In [14]:
# Step 1: Pivot the dataset to wide format by year for selected latent variables
columns_to_pivot = [
    "accommodation_density_score",
    "temporary_dwelling_index"
]

df_accommodation_latents_wide = df_accommodation_latents_final.pivot(
    index=["geography", "geography code"],
    columns="date",
    values=columns_to_pivot
).reset_index()

# Step 2: Flatten multi-level columns and format them as feature_year
df_accommodation_latents_wide.columns = [
    f"{col[0]}_{col[1]}" if col[1] in [2011, 2021] else col[0]
    for col in df_accommodation_latents_wide.columns
]

# Step 3: Display dataset shape and preview
print(f"\n[df_accommodation_latents_wide] shape: {df_accommodation_latents_wide.shape}\n")
display(
    df_accommodation_latents_wide.head()
    .style
    .hide(axis="index")
    .set_properties(**{"text-align": "left"})
    .set_table_styles([{
        "selector": "th",
        "props": [("text-align", "left")]
    }])
)


# Step 4: Check and report missing values if present
total_missing = df_accommodation_latents_wide.isna().sum().sum()
if total_missing:
    print(f"\n[df_accommodation_latents_wide] Total missing values: {total_missing}")
    missing_pct = (
        df_accommodation_latents_wide.isna().mean().mul(100).round(2)
    )
    print(
        "[df_accommodation_latents_wide] Missing % by column (non-zero only):\n",
        missing_pct[missing_pct > 0].astype(str) + " %"
    )
else:
    print(f"\n[df_accommodation_latents_wide] Total missing values: 0")
    print("[df_accommodation_latents_wide] No missing values found.")

# Step 5: Save output to CSV format for Tableau visualization
output_dir = "Dashboard3"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Accommodation_Type_Wide_Format_2011_2021.csv")
if os.path.exists(output_path):
    print(f"\n[SKIPPED] File already exists: {output_path}")
else:
    df_accommodation_latents_wide.to_csv(output_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_path} (Ready for Tableau Visualization).")


[df_accommodation_latents_wide] shape: (348, 6)



geography,geography code,accommodation_density_score_2011,accommodation_density_score_2021,temporary_dwelling_index_2011,temporary_dwelling_index_2021
Adur,E07000223,-0.329505,-0.431038,-0.099680,-0.163506
Allerdale,E07000026,-0.132828,-0.374697,-0.121306,-0.178638
Amber Valley,E07000032,-0.135390,-0.364866,-0.206491,-0.237700
Arun,E07000224,0.024073,-0.197913,0.486606,0.325536
Ashfield,E07000170,-0.152792,-0.426863,-0.274797,-0.292576



[df_accommodation_latents_wide] Total missing values: 0
[df_accommodation_latents_wide] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard3\England_Wales_SocioEconomic_Accommodation_Type_Wide_Format_2011_2021.csv (Ready for Tableau Visualization).


<a id="subsection-25-bayesian-cfa-of-tenure-structure-2011-vs-2021-modeling"></a>
#### 2.5: Bayesian CFA of Tenure Structure (2011 vs 2021 Modeling)

In this step, we estimated two tenure-related latent constructs—home ownership stability and rental insecurity—by applying Bayesian confirmatory factor analysis (CFA) to harmonized tenure datasets from 2011 and 2021.

Following standardization and model construction, we fitted the CFA model with variational inference, extracted latent estimates and uncertainty bounds, and merged the results back to the combined dataset. The output included the full latent structure with year-specific values, ready for comparative interpretation.

In [15]:
# Suppress convergence messages during variational inference for clean output
warnings.filterwarnings("ignore", category=UserWarning)
for name in ["pytensor", "pytensor.configdefaults", "pymc"]:
    logging.getLogger(name).setLevel(logging.ERROR)

# Step 1: Define latent variable structure using an indicator map for tenure-related features
indicator_map = {
    "home_ownership_stability": [
        "tenure_of_household_owned_owns_outright",
        "tenure_of_household_owned_owns_with_a_mortgage_or_loan",
    ],
    "rental_insecurity_score": [
        "tenure_of_household_private_rented_private_landlord_or_letting_agency",
        "tenure_of_household_lives_rent_free",
        "tenure_of_household_shared_ownership",
    ],
}

# Step 2: Define a function to run Bayesian confirmatory factor analysis
def run_bayesian_cfa(df_final: pd.DataFrame, indicator_map: dict) -> pd.DataFrame:
    latent_names = list(indicator_map.keys())

    # Collect all unique indicator columns
    chosen_cols = []
    for latent in latent_names:
        for col in indicator_map[latent]:
            if col not in chosen_cols:
                chosen_cols.append(col)

    # Standardize indicator variables for modeling
    df_model = df_final[chosen_cols].copy()
    if df_model.columns.duplicated().any():
        df_model = df_model.loc[:, ~df_model.columns.duplicated()].copy()

    means = df_model.mean()
    stds = df_model.std().replace(0, 1e-6)
    df_model_std = (df_model - means) / stds

    Y = df_model_std.values
    n_obs, n_ind = Y.shape
    n_latent = len(latent_names)

    # Create an index mapping each indicator to its latent construct
    ind2latent_idx = np.array([
        next(
            lat_idx
            for lat_idx, lat in enumerate(latent_names)
            if col in indicator_map[lat]
        )
        for col in df_model.columns
    ])

    coords = {
        "obs": range(n_obs),
        "indicator": df_model.columns,
        "latent": latent_names,
    }

    # Build and fit the Bayesian CFA model using PyMC and ADVI
    with pm.Model(coords=coords) as cfa_model:
        F = pm.Normal("F", 0.0, 1.0, dims=("obs", "latent"))
        alpha = pm.Normal("alpha", 0.0, 1.0, dims="indicator")
        lambda_raw = pm.HalfNormal("lambda_raw", 1.0, dims="indicator")

        Lambda = pm.Deterministic(
            "Lambda",
            pm.math.stack(
                [pm.math.switch(ind2latent_idx == k, lambda_raw, 0.0)
                 for k in range(n_latent)]
            ).T,
        )

        sigma = pm.HalfNormal("sigma", 1.0, dims="indicator")
        mu = alpha + pm.math.dot(F, Lambda.T)

        pm.Normal("Y_obs", mu, sigma, observed=Y, dims=("obs", "indicator"))

        with contextlib.redirect_stdout(io.StringIO()), \
             contextlib.redirect_stderr(io.StringIO()):
            approx = pm.fit(
                method="advi",
                n=6_000,     
                callbacks=[SilentConvergence(tolerance=1e-4)],
                progressbar=False,
                random_seed=42,
            )
            trace = approx.sample(draws=2_000, random_seed=42)   

    posterior = trace.posterior.stack(sample=("chain", "draw"))

    # Extract posterior mean and uncertainty intervals for each latent factor
    for lat in latent_names:
        da = posterior["F"].sel(latent=lat)
        df_model[f"{lat}_mean"] = da.mean(dim="sample").values
        df_model[f"{lat}_hdi_2_5"] = np.percentile(da.values, 2.5, axis=1)
        df_model[f"{lat}_hdi_97_5"] = np.percentile(da.values, 97.5, axis=1)

    # Merge the inferred scores back into the original dataset
    df_enriched = df_final.merge(
        df_model[[c for c in df_model.columns if "_mean" in c or "_hdi" in c]],
        left_index=True,
        right_index=True,
        how="left",
    )

    return df_enriched

# Step 3: Apply the CFA model to the combined tenure dataset
df_with_tenure_latents = run_bayesian_cfa(
    df_combined_tenure,
    indicator_map=indicator_map
)

# Step 4: Prepare the final list of selected columns for export
base_cols = ["date", "geography", "geography code"]
all_selected_cols = base_cols.copy()
for latent, indicators in indicator_map.items():
    all_selected_cols.extend(indicators)
    all_selected_cols.append(f"{latent}_mean")
    all_selected_cols.append(f"{latent}_hdi_2_5")
    all_selected_cols.append(f"{latent}_hdi_97_5")
all_selected_cols = list(dict.fromkeys(all_selected_cols))

# Step 5: Build the output DataFrame and sort by geography
df_tenure_latents_final = df_with_tenure_latents[all_selected_cols].copy()
df_tenure_latents_final = df_tenure_latents_final.sort_values(by="geography code")

# Step 6: Rename columns for clarity in final output
rename_dict = {
    "home_ownership_stability_mean": "home_ownership_stability",
    "home_ownership_stability_hdi_2_5": "home_ownership_stability_lower_2_5",
    "home_ownership_stability_hdi_97_5": "home_ownership_stability_upper_97_5",
    "rental_insecurity_score_mean": "rental_insecurity_score",
    "rental_insecurity_score_hdi_2_5": "rental_insecurity_score_lower_2_5",
    "rental_insecurity_score_hdi_97_5": "rental_insecurity_score_upper_97_5"
}
df_tenure_latents_final = df_tenure_latents_final.rename(columns=rename_dict)

print(f"\n[df_tenure_latents_final] shape: {df_tenure_latents_final.shape}\n")

# Step 7: Display a year-separated preview of the final dataset if available
if not set(df_tenure_latents_final["date"]).issuperset({2011, 2021}):
    print("[WARN] Cannot generate year-separated preview: 2011 or 2021 data missing.")
else:
    df11_preview = (
        df_tenure_latents_final[
            df_tenure_latents_final["date"] == 2011
        ].head()
    )
    df21_preview = (
        df_tenure_latents_final[
            df_tenure_latents_final["date"] == 2021
        ].head()
    )
    ellipsis_row = pd.DataFrame(
        [["..."] * df_tenure_latents_final.shape[1]],
        columns=df_tenure_latents_final.columns
    )

    df_preview = pd.concat([
        df11_preview,
        ellipsis_row,
        df21_preview,
        ellipsis_row,
    ], ignore_index=True)

    display(
        df_preview
        .style
        .hide(axis="index")
        .set_properties(**{"text-align": "left"})
        .set_table_styles([{
            "selector": "th",
            "props": [("text-align", "left")]
        }])
    )

# Step 8: Report missing values across the final dataset
total_missing = df_tenure_latents_final.isna().sum().sum()
if total_missing:
    print(f"\n[df_tenure_latents_final] Total missing values: {total_missing}")
    missing_pct = (
        df_tenure_latents_final.isna().mean().mul(100).round(2)
    )
    print(
        "[df_tenure_latents_final] Missing % by column (non-zero only):\n",
        missing_pct[missing_pct > 0].astype(str) + " %"
    )
else:
    print(f"\n[df_tenure_latents_final] Total missing values: 0")
    print("[df_tenure_latents_final] No missing values found.")

# Step 9: Save output to CSV format for Tableau visualization
output_dir = "Dashboard4"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Tenure_2011_2021.csv")
if os.path.exists(output_path):
    print(f"\n[SKIPPED] File already exists: {output_path}")
else:
    df_tenure_latents_final.to_csv(output_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_path} (Ready for Tableau Visualization).")


[df_tenure_latents_final] shape: (696, 14)



date,geography,geography code,tenure_of_household_owned_owns_outright,tenure_of_household_owned_owns_with_a_mortgage_or_loan,home_ownership_stability,home_ownership_stability_lower_2_5,home_ownership_stability_upper_97_5,tenure_of_household_private_rented_private_landlord_or_letting_agency,tenure_of_household_lives_rent_free,tenure_of_household_shared_ownership,rental_insecurity_score,rental_insecurity_score_lower_2_5,rental_insecurity_score_upper_97_5
2011,Hartlepool,E06000001,26,13104,-0.349920,-1.933873,1.141225,5525,71,349,-0.354061,-1.756820,1.078473
2011,Middlesbrough,E06000002,98,17523,-0.269014,-1.853871,1.287512,8648,193,570,-0.052286,-1.452053,1.408103
2011,Redcar and Cleveland,E06000003,55,19899,-0.266229,-1.814742,1.291218,6698,103,578,-0.222247,-1.592419,1.162701
2011,Stockton-on-Tees,E06000004,41,30940,-0.058489,-1.623144,1.516195,9417,141,788,-0.027154,-1.469557,1.333610
2011,Darlington,E06000005,112,16479,-0.311776,-1.927212,1.226424,7758,97,546,-0.215086,-1.620123,1.236109
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Hartlepool,E06000001,12761,11073,-0.194657,-1.760502,1.350086,6338,68,239,-0.348090,-1.740728,1.077640
2021,Middlesbrough,E06000002,17224,15913,-0.047692,-1.573862,1.451520,11286,115,330,-0.216712,-1.713127,1.283540
2021,Redcar and Cleveland,E06000003,22649,17205,0.022992,-1.486383,1.572177,8410,41,346,-0.329396,-1.710380,1.099628
2021,Stockton-on-Tees,E06000004,27726,27686,0.260084,-1.252401,1.841596,12388,79,385,-0.152241,-1.543223,1.194568



[df_tenure_latents_final] Total missing values: 0
[df_tenure_latents_final] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard4\England_Wales_SocioEconomic_Tenure_2011_2021.csv (Ready for Tableau Visualization).


<a id="subsection-26-pivot-tenure-latent-scores-into-year-wide-format-for-2011–2021-comparison"></a>
#### 2.6: Pivot Tenure Latent Scores into Year-Wide Format for 2011–2021 Comparison

In this step, we reshaped the tenure latent scores to a wide format that enabled side-by-side comparison between 2011 and 2021 values. We pivoted home ownership stability and rental insecurity into separate columns per year, ensured column alignment, and handled any missing data. The final dataset was saved in a structured format suitable for integration into a visualization dashboard.

In [16]:
# Step 1: Pivot the dataset to wide format by year for selected tenure-based latent variables
columns_to_pivot = [
    "home_ownership_stability",
    "rental_insecurity_score"
]

df_tenure_latents_wide = df_tenure_latents_final.pivot(
    index=["geography", "geography code"],
    columns="date",
    values=columns_to_pivot
).reset_index()

# Step 2: Flatten multi-level columns and format them as feature_year
df_tenure_latents_wide.columns = [
    f"{col[0]}_{col[1]}" if col[1] in [2011, 2021] else col[0]
    for col in df_tenure_latents_wide.columns
]

# Step 3: Display dataset shape and preview
print(f"\n[df_tenure_latents_wide] shape: {df_tenure_latents_wide.shape}\n")
display(
    df_tenure_latents_wide.head()
    .style
    .hide(axis="index")
    .set_properties(**{"text-align": "left"})
    .set_table_styles([{
        "selector": "th",
        "props": [("text-align", "left")]
    }])
)


# Step 4: Check and report missing values if present
total_missing = df_tenure_latents_wide.isna().sum().sum()
if total_missing:
    print(f"\n[df_tenure_latents_wide] Total missing values: {total_missing}")
    missing_pct = (
        df_tenure_latents_wide.isna().mean().mul(100).round(2)
    )
    print(
        "[df_tenure_latents_wide] Missing % by column (non-zero only):\n",
        missing_pct[missing_pct > 0].astype(str) + " %"
    )
else:
    print(f"\n[df_tenure_latents_wide] Total missing values: 0")
    print("[df_tenure_latents_wide] No missing values found.")

# Step 5: Save output to CSV format for Tableau visualization
output_dir = "Dashboard4"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Tenure_Wide_Format_2011_2021.csv")
if os.path.exists(output_path):
    print(f"\n[SKIPPED] File already exists: {output_path}")
else:
    df_tenure_latents_wide.to_csv(output_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_path} (Ready for Tableau Visualization).")


[df_tenure_latents_wide] shape: (348, 6)



geography,geography code,home_ownership_stability_2011,home_ownership_stability_2021,rental_insecurity_score_2011,rental_insecurity_score_2021
Adur,E07000223,-0.407244,-0.256914,-0.523139,-0.509361
Allerdale,E07000026,-0.301917,-0.108874,-0.348809,-0.484113
Amber Valley,E07000032,-0.208463,0.084257,-0.288111,-0.355177
Arun,E07000224,-0.252468,0.220258,-0.122259,-0.060042
Ashfield,E07000170,-0.233222,-0.037263,-0.350679,-0.313106



[df_tenure_latents_wide] Total missing values: 0
[df_tenure_latents_wide] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard4\England_Wales_SocioEconomic_Tenure_Wide_Format_2011_2021.csv (Ready for Tableau Visualization).


<a id="section-3-dimensionality-reduction-and-clustering-of-socio-economic-patterns"></a>
### Part 3: Dimensionality Reduction and Clustering of Socio-Economic Patterns

In this part, we performed dimensionality reduction and clustering on both observed and latent socio-economic features to support spatial pattern analysis and visual exploration.

We first applied PCA followed by t-SNE to a selected set of raw 2011 indicators (3.1), and separately projected latent socio-economic constructs using UMAP (3.2). Next, we applied UMAP and K-Means clustering to accommodation-related indicators across both 2011 and 2021 (3.3), then repeated the process for tenure-based features (3.4). All projections were standardized and clustered independently by year, and cluster-wise profiles were computed and merged into the final outputs. The resulting datasets were exported for Tableau-based comparative visualization.

<a id="subsection-31-pca-––-t-sne-projection-of-observed-socio-economic-indicators-2011"></a>
#### 3.1: PCA –– t-SNE Projection of Observed Socio-Economic Indicators (2011)

In this step, we applied dimensionality reduction to a curated set of socio-economic and housing-related indicators from the 2011 census using a two-stage approach: Principal Component Analysis (PCA) followed by t-distributed Stochastic Neighbor Embedding (t-SNE). The original variables were standardized, reduced to four principal components, and then embedded into two dimensions using t-SNE. K-Means clustering (k=5) was performed on the embedded space, and the resulting cluster labels were appended to the projection metadata. The final dataset, including projections and cluster labels, was exported as a CSV file for visualization.

In [17]:
# Step 1 Select indicator columns for dimensionality reduction
indicator_cols = [
    "social_rented_total",
    "social_rented_rented_from_council_local_authority",
    "occupancy_rating_bedrooms_of_0",
    "unshared_dwelling_flat_maisonette_or_apartment_part_of_a_converted_or_shared_house_including_bed_sits",
    "occupancy_rating_bedrooms_of_minus_1",
    "unshared_dwelling_caravan_or_other_mobile_or_temporary_structure",
    "private_rented_total",
    "shared_ownership_part_owned_and_part_rented",
    "private_rented_relative_or_friend_of_household_member",
    "no_cars_or_vans_in_household",
    "1_car_or_van_in_household",
    "sum_of_all_cars_or_vans_in_the_area"
]

# Step 2: Standardize input features before projection
X = df_2011_with_latents[indicator_cols].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Apply PCA to reduce feature space before t-SNE
pca = PCA(n_components=4, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Step 4: Apply t-SNE for 2D embedding of PCA-transformed features
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, random_state=42)
X_tsne = tsne.fit_transform(X_pca)

# Step 5: Prepare metadata and append t-SNE coordinates
df_2011_pca_tsne = df_2011_with_latents[["date", "geography", "geography code"]].copy()
df_2011_pca_tsne["TSNE1"] = X_tsne[:, 0]
df_2011_pca_tsne["TSNE2"] = X_tsne[:, 1]

# Step 6: Perform KMeans clustering on the t-SNE projection
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(df_2011_pca_tsne[["TSNE1", "TSNE2"]])
cluster_labels = kmeans.labels_ + 1
df_2011_pca_tsne["Cluster Label"] = cluster_labels

# Step 7: Display shape and preview of results
print(f"\n[df_2011_pca_tsne] shape: {df_2011_pca_tsne.shape}\n")
left_align_cols = df_2011_pca_tsne.columns[:-1]
display(
    df_2011_pca_tsne.head()
    .style
    .hide(axis="index")
    .set_properties(subset=left_align_cols, **{"text-align": "left"})
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }])
)

# Step 8: Check and report missing values if any
total_missing = df_2011_pca_tsne.isna().sum().sum()
if total_missing:
    print(f"\n[df_2011_pca_tsne] Total missing values: {total_missing}")
    missing_pct = df_2011_pca_tsne.isna().mean().mul(100).round(2)
    print("[df_2011_pca_tsne] Missing % by column (non-zero only):\n",
          missing_pct[missing_pct > 0].astype(str) + " %")
else:
    print(f"\n[df_2011_pca_tsne] Total missing values: 0")
    print("[df_2011_pca_tsne] No missing values found.")

# Step 9: Select output columns and finalize DataFrame
df_2011_pca_tsne = df_2011_pca_tsne[[
    "date",
    "geography",
    "geography code",
    "TSNE1",
    "TSNE2",
    "Cluster Label"
]]

# Step 10: Save output to CSV format for Tableau visualization
output_dir = "Dashboard2"
os.makedirs(output_dir, exist_ok=True)

output_tsne_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Housing_PCA_TSNE_2011.csv")
if os.path.exists(output_tsne_path):
    print(f"\n[SKIPPED] File already exists: {output_tsne_path}")
else:
    df_2011_pca_tsne.to_csv(output_tsne_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_tsne_path} (Ready for Tableau Visualization).")


[df_2011_pca_tsne] shape: (348, 6)



date,geography,geography code,TSNE1,TSNE2,Cluster Label
2011,Hartlepool,E06000001,13.621572,5.965090,5
2011,Middlesbrough,E06000002,1.467700,-6.495614,4
2011,Redcar and Cleveland,E06000003,0.781263,-7.087644,4
2011,Stockton-on-Tees,E06000004,-3.910678,-6.886680,4
2011,Darlington,E06000005,10.864006,0.663811,5



[df_2011_pca_tsne] Total missing values: 0
[df_2011_pca_tsne] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard2\England_Wales_SocioEconomic_Housing_PCA_TSNE_2011.csv (Ready for Tableau Visualization).


<a id="subsection-32-umap-projection-of-latent-socio-economic-dimensions-2011"></a>
#### 3.2: UMAP Projection of Latent Socio-Economic Dimensions (2011)

In this step, we used Uniform Manifold Approximation and Projection (UMAP) to visualize four latent socio-economic dimensions derived from Bayesian confirmatory factor analysis (CFA) applied to the 2011 dataset. After z-score standardization, the latent features were projected into a two-dimensional embedding space. K-Means clustering (k=5) was then applied to identify structural groupings within the embedding. The final UMAP coordinates and cluster labels were combined with metadata and exported for downstream visual analytics.

In [18]:
# Step 1 Select latent socio-economic index variables
X_latent = df_2011_with_latents[[
    "latent_deprivation_index",
    "housing_strain_score",
    "tenure_instability_score",
    "low_mobility_access_index"
]].copy()

# Step 2 Standardize latent features before UMAP
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_latent)

# Step 3 Apply UMAP for non-linear dimensionality reduction
reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42)
X_umap = reducer.fit_transform(X_scaled)

# Step 4 Build DataFrame with metadata and UMAP coordinates
df_2011_umap = df_2011_with_latents[["date", "geography", "geography code"]].copy()
df_2011_umap["UMAP1"] = X_umap[:, 0]
df_2011_umap["UMAP2"] = X_umap[:, 1]

# Step 5 Perform clustering on the UMAP-projected data
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(df_2011_umap[["UMAP1", "UMAP2"]])
df_2011_umap["Cluster Label"] = kmeans.labels_ + 1

# Step 6 Display shape and preview of results
print(f"\n[df_2011_umap] shape: {df_2011_umap.shape}\n")
left_align_cols = df_2011_umap.columns[:-1]

display(
    df_2011_umap.head()
    .style
    .hide(axis="index")
    .set_properties(subset=left_align_cols, **{"text-align": "left"})
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }])
)

# Step 7 Check and report missing values if any
total_missing = df_2011_umap.isna().sum().sum()
if total_missing:
    print(f"\n[df_2011_umap] Total missing values: {total_missing}")
    missing_pct = df_2011_umap.isna().mean().mul(100).round(2)
    print("[df_2011_umap] Missing % by column (non-zero only):\n",
          missing_pct[missing_pct > 0].astype(str) + " %")
else:
    print(f"\n[df_2011_umap] Total missing values: 0")
    print("[df_2011_umap] No missing values found.")

# Step 8: Save output to CSV format for Tableau visualization
output_dir = "Dashboard2"
os.makedirs(output_dir, exist_ok=True)

output_umap_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Housing_UMAP_2011.csv")
if os.path.exists(output_umap_path):
    print(f"\n[SKIPPED] File already exists: {output_umap_path}")
else:
    df_2011_umap.to_csv(output_umap_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_umap_path} (Ready for Tableau Visualization).")


[df_2011_umap] shape: (348, 6)



date,geography,geography code,UMAP1,UMAP2,Cluster Label
2011,Hartlepool,E06000001,9.408449,3.636358,5
2011,Middlesbrough,E06000002,5.298033,5.571430,3
2011,Redcar and Cleveland,E06000003,5.481946,5.444964,3
2011,Stockton-on-Tees,E06000004,3.725358,6.736191,3
2011,Darlington,E06000005,8.964059,3.841808,5



[df_2011_umap] Total missing values: 0
[df_2011_umap] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard2\England_Wales_SocioEconomic_Housing_UMAP_2011.csv (Ready for Tableau Visualization).


<a id="subsection-33-umap-projection-and-clustering-of-accommodation-features-2011-vs-2021"></a>
#### 3.3: UMAP Projection and Clustering of Accommodation Features (2011 vs 2021)

In this step, we performed dimensionality reduction on selected accommodation-related features from the 2011 and 2021 datasets using UMAP. We then applied K-Means clustering separately for each year with k=5. Cluster labels were merged into the metadata, and each cluster’s composition was computed by aggregating and normalizing feature proportions. The resulting cluster profiles were appended to the dataset and exported as a CSV file for Tableau visualization.

In [19]:
# Step 1: Select accommodation-related features for dimensionality reduction and clustering
features = [
    "accommodation_type_terraced",
    "accommodation_type_part_of_a_converted_or_shared_house_including_bedsits",
    "accommodation_type_in_a_purpose_built_block_of_flats_or_tenement",
    "accommodation_type_a_caravan_or_other_mobile_or_temporary_structure",
    "accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop"
]

# Step 2: Standardize selected indicators for embedding
X = df_accommodation_latents_final[features].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Apply UMAP for 2D dimensionality reduction
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric="euclidean", random_state=42)
X_umap = reducer.fit_transform(X_scaled)

# Step 4: Prepare metadata and append UMAP coordinates
df_accommodation_type_2011_2021_umap = df_accommodation_latents_final[["date", "geography", "geography code"]].copy()
df_accommodation_type_2011_2021_umap["UMAP1"] = X_umap[:, 0]
df_accommodation_type_2011_2021_umap["UMAP2"] = X_umap[:, 1]

# Step 5 Split UMAP results into 2011 and 2021 subsets
df_umap_2011 = df_accommodation_type_2011_2021_umap[df_accommodation_type_2011_2021_umap["date"] == 2011].copy()
df_umap_2021 = df_accommodation_type_2011_2021_umap[df_accommodation_type_2011_2021_umap["date"] == 2021].copy()

# Step 6 Apply KMeans clustering independently for each year
kmeans_2011 = KMeans(n_clusters=5, random_state=42)
df_umap_2011["Cluster Label"] = kmeans_2011.fit_predict(df_umap_2011[["UMAP1", "UMAP2"]]) + 1

kmeans_2021 = KMeans(n_clusters=5, random_state=42)
df_umap_2021["Cluster Label"] = kmeans_2021.fit_predict(df_umap_2021[["UMAP1", "UMAP2"]]) + 1

# Step 7 Combine the clustered results for both years
df_accommodation_type_2011_2021_umap = pd.concat([df_umap_2011, df_umap_2021], ignore_index=True)

# Step 8: Join cluster labels to original feature data
df_full = df_accommodation_latents_final[["date", "geography", "geography code"] + features].copy()
df_full["Cluster Label"] = df_accommodation_type_2011_2021_umap["Cluster Label"]

# Step 9: Aggregate feature totals by cluster and year
cluster_sums = (
    df_full
    .groupby(["date", "Cluster Label"])[features]
    .sum()
    .reset_index()
)

# Step 10: Convert feature sums to within-cluster proportions
cluster_totals = cluster_sums[features].sum(axis=1)
cluster_prop = cluster_sums[features].div(cluster_totals, axis=0)

# Step 11: Normalize proportions across clusters using min-max scaling
minmax_scaler = MinMaxScaler()
cluster_prop_norm = pd.DataFrame(
    minmax_scaler.fit_transform(cluster_prop),
    columns=[f"{col}_relative_score" for col in features]
)

# Step 12: Construct final cluster profile table
cluster_profiles = pd.concat([
    cluster_sums[["date", "Cluster Label"]].reset_index(drop=True),
    cluster_prop_norm
], axis=1)

# Step 13: Merge profile scores back into the original UMAP table
df_accommodation_type_2011_2021_umap = df_accommodation_type_2011_2021_umap.merge(
    cluster_profiles,
    on=["date", "Cluster Label"],
    how="left"
)

# Step 14: Display dataframe shape and preview
print(f"\n[df_accommodation_type_2011_2021_umap] shape: {df_accommodation_type_2011_2021_umap.shape}\n")

if not set(df_accommodation_type_2011_2021_umap["date"]).issuperset({2011, 2021}):
    print("[WARN] Cannot generate year-separated preview: 2011 or 2021 data missing.")
else:
    df11_preview = (
        df_accommodation_type_2011_2021_umap[
            df_accommodation_type_2011_2021_umap["date"] == 2011
        ]
        .sort_values("geography code")
        .head()
    )
    df21_preview = (
        df_accommodation_type_2011_2021_umap[
            df_accommodation_type_2011_2021_umap["date"] == 2021
        ]
        .sort_values("geography code")
        .head()
    )
    ellipsis_row = pd.DataFrame(
        [["..."] * df_accommodation_type_2011_2021_umap.shape[1]],
        columns=df_accommodation_type_2011_2021_umap.columns
    )
    df_preview = pd.concat(
        [df11_preview, ellipsis_row, df21_preview, ellipsis_row],
        ignore_index=True
    )
    display(
        df_preview
        .style
        .hide(axis="index")
        .set_properties(**{"text-align": "left"})
        .set_table_styles([{
            "selector": "th",
            "props": [("text-align", "left")]
        }])
    )

# Step 15: Check and report missing values
total_missing = df_accommodation_type_2011_2021_umap.isna().sum().sum()
if total_missing:
    print(f"\n[df_accommodation_type_2011_2021_umap] Total missing values: {total_missing}")
    missing_pct = df_accommodation_type_2011_2021_umap.isna().mean().mul(100).round(2)
    print("[df_accommodation_type_2011_2021_umap] Missing % by column (non-zero only):\n",
          missing_pct[missing_pct > 0].astype(str) + " %")
else:
    print(f"\n[df_accommodation_type_2011_2021_umap] Total missing values: 0")
    print("[df_accommodation_type_2011_2021_umap] No missing values found.")

# Step 16: Save output to CSV format for Tableau visualization
output_dir = "Dashboard5"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Accommodation_Type_UMAP_2011_2021.csv")
if os.path.exists(output_path):
    print(f"\n[SKIPPED] File already exists: {output_path}")
else:
    df_accommodation_type_2011_2021_umap.to_csv(output_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_path} (Ready for Tableau Visualization).")


[df_accommodation_type_2011_2021_umap] shape: (696, 11)



date,geography,geography code,UMAP1,UMAP2,Cluster Label,accommodation_type_terraced_relative_score,accommodation_type_part_of_a_converted_or_shared_house_including_bedsits_relative_score,accommodation_type_in_a_purpose_built_block_of_flats_or_tenement_relative_score,accommodation_type_a_caravan_or_other_mobile_or_temporary_structure_relative_score,accommodation_type_in_a_commercial_building_for_example_in_an_office_building_hotel_or_over_a_shop_relative_score
2011,Hartlepool,E06000001,13.544476,8.167521,5,0.041079,0.888718,0.096105,0.028923,0.742083
2011,Middlesbrough,E06000002,12.581448,6.810639,5,0.041079,0.888718,0.096105,0.028923,0.742083
2011,Redcar and Cleveland,E06000003,12.573186,7.007524,5,0.041079,0.888718,0.096105,0.028923,0.742083
2011,Stockton-on-Tees,E06000004,12.386235,6.283976,5,0.041079,0.888718,0.096105,0.028923,0.742083
2011,Darlington,E06000005,13.469326,8.015670,5,0.041079,0.888718,0.096105,0.028923,0.742083
...,...,...,...,...,...,...,...,...,...,...
2021,Hartlepool,E06000001,-0.751377,3.959010,5,1.000000,0.015724,0.672675,0.110049,0.001993
2021,Middlesbrough,E06000002,-1.129311,3.367649,5,1.000000,0.015724,0.672675,0.110049,0.001993
2021,Redcar and Cleveland,E06000003,-0.804460,3.647670,5,1.000000,0.015724,0.672675,0.110049,0.001993
2021,Stockton-on-Tees,E06000004,-1.240769,3.250787,5,1.000000,0.015724,0.672675,0.110049,0.001993



[df_accommodation_type_2011_2021_umap] Total missing values: 0
[df_accommodation_type_2011_2021_umap] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard5\England_Wales_SocioEconomic_Accommodation_Type_UMAP_2011_2021.csv (Ready for Tableau Visualization).


<a id="subsection-34-umap-projection-and-clustering-of-tenure-features-2011-vs-2021"></a>
#### 3.4: UMAP Projection and Clustering of Tenure Features (2011 vs 2021)

In this step, we applied UMAP to reduce the dimensionality of tenure-related indicators from both the 2011 and 2021 datasets. K-Means clustering with k=5 was performed independently for each year. Cluster assignments were merged with the metadata, and within-cluster feature distributions were aggregated and min-max normalized. The enriched dataset, containing UMAP coordinates and cluster profiles, was exported in CSV format for Tableau visualization.

In [20]:
# Step 1: Select tenure-related indicators for dimensionality reduction and clustering
features = [
    "tenure_of_household_owned_owns_outright",
    "tenure_of_household_owned_owns_with_a_mortgage_or_loan",
    "tenure_of_household_private_rented_private_landlord_or_letting_agency",
    "tenure_of_household_lives_rent_free",
    "tenure_of_household_shared_ownership"
]

# Step 2: Standardize selected indicators for embedding
X = df_tenure_latents_final[features].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Apply UMAP for 2D dimensionality reduction
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric="euclidean", random_state=42)
X_umap = reducer.fit_transform(X_scaled)

# Step 4: Create base DataFrame with metadata and UMAP projection
df_tenure_2011_2021_umap = df_tenure_latents_final[["date", "geography", "geography code"]].copy()
df_tenure_2011_2021_umap["UMAP1"] = X_umap[:, 0]
df_tenure_2011_2021_umap["UMAP2"] = X_umap[:, 1]

# Step 5: Split UMAP results into 2011 and 2021 subsets
df_umap_2011 = df_tenure_2011_2021_umap[df_tenure_2011_2021_umap["date"] == 2011].copy()
df_umap_2021 = df_tenure_2011_2021_umap[df_tenure_2011_2021_umap["date"] == 2021].copy()

# Step 6: Apply KMeans clustering independently for each year
kmeans_2011 = KMeans(n_clusters=5, random_state=42)
df_umap_2011["Cluster Label"] = kmeans_2011.fit_predict(df_umap_2011[["UMAP1", "UMAP2"]]) + 1

kmeans_2021 = KMeans(n_clusters=5, random_state=42)
df_umap_2021["Cluster Label"] = kmeans_2021.fit_predict(df_umap_2021[["UMAP1", "UMAP2"]]) + 1

# Step 7: Combine the clustered results for both years
df_tenure_2011_2021_umap = pd.concat([df_umap_2011, df_umap_2021], ignore_index=True)

# Step 8: Link cluster labels back to full feature values for profiling
df_full = df_tenure_latents_final[["date", "geography", "geography code"] + features].copy()
df_full["Cluster Label"] = df_tenure_2011_2021_umap["Cluster Label"]

# Step 9: Calculate total feature values by cluster and year
cluster_sums = (
    df_full
    .groupby(["date", "Cluster Label"])[features]
    .sum()
    .reset_index()
)

# Step 10: Normalize feature proportions within each cluster
cluster_totals = cluster_sums[features].sum(axis=1)
cluster_prop = cluster_sums[features].div(cluster_totals, axis=0)

# Step 11: Scale proportions using MinMax scaling for comparability
minmax_scaler = MinMaxScaler()
cluster_prop_norm = pd.DataFrame(
    minmax_scaler.fit_transform(cluster_prop),
    columns=[f"{col}_relative_score" for col in features]
)

# Step 12: Combine scaled proportions with cluster metadata
cluster_profiles = pd.concat([
    cluster_sums[["date", "Cluster Label"]].reset_index(drop=True),
    cluster_prop_norm
], axis=1)

# Step 13: Join cluster profiles back to UMAP coordinate data
df_tenure_2011_2021_umap = df_tenure_2011_2021_umap.merge(
    cluster_profiles,
    on=["date", "Cluster Label"],
    how="left"
)

# Step 14: Display dataframe shape and preview
print(f"\n[df_tenure_2011_2021_umap] shape: {df_tenure_2011_2021_umap.shape}\n")

if not set(df_tenure_2011_2021_umap["date"]).issuperset({2011, 2021}):
    print("[WARN] Cannot generate year-separated preview: 2011 or 2021 data missing.")
else:
    df11_preview = (
        df_tenure_2011_2021_umap[
            df_tenure_2011_2021_umap["date"] == 2011
        ]
        .sort_values("geography code")
        .head()
    )
    df21_preview = (
        df_tenure_2011_2021_umap[
            df_tenure_2011_2021_umap["date"] == 2021
        ]
        .sort_values("geography code")
        .head()
    )
    ellipsis_row = pd.DataFrame(
        [["..."] * df_tenure_2011_2021_umap.shape[1]],
        columns=df_tenure_2011_2021_umap.columns
    )
    df_preview = pd.concat(
        [df11_preview, ellipsis_row, df21_preview, ellipsis_row],
        ignore_index=True
    )
    display(
        df_preview
        .style
        .hide(axis="index")
        .set_properties(**{"text-align": "left"})
        .set_table_styles([{
            "selector": "th",
            "props": [("text-align", "left")]
        }])
    )

# Step 15: Check and report missing values
total_missing = df_tenure_2011_2021_umap.isna().sum().sum()
if total_missing:
    print(f"\n[df_tenure_2011_2021_umap] Total missing values: {total_missing}")
    missing_pct = df_tenure_2011_2021_umap.isna().mean().mul(100).round(2)
    print("[df_tenure_2011_2021_umap] Missing % by column (non-zero only):\n",
          missing_pct[missing_pct > 0].astype(str) + " %")
else:
    print(f"\n[df_tenure_2011_2021_umap] Total missing values: 0")
    print("[df_tenure_2011_2021_umap] No missing values found.")

# Step 16: Save output to CSV format for Tableau visualization
output_dir = "Dashboard6"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "England_Wales_SocioEconomic_Tenure_UMAP_2011_2021.csv")
if os.path.exists(output_path):
    print(f"\n[SKIPPED] File already exists: {output_path}")
else:
    df_tenure_2011_2021_umap.to_csv(output_path, index=False)
    print(f"\n[SAVED] The DataFrame has been exported to: {output_path} (Ready for Tableau Visualization).")


[df_tenure_2011_2021_umap] shape: (696, 11)



date,geography,geography code,UMAP1,UMAP2,Cluster Label,tenure_of_household_owned_owns_outright_relative_score,tenure_of_household_owned_owns_with_a_mortgage_or_loan_relative_score,tenure_of_household_private_rented_private_landlord_or_letting_agency_relative_score,tenure_of_household_lives_rent_free_relative_score,tenure_of_household_shared_ownership_relative_score
2011,Hartlepool,E06000001,-3.089316,1.866667,3,0.001252,0.969730,0.368865,0.765343,0.838150
2011,Middlesbrough,E06000002,-0.035549,2.681257,5,0.000000,0.926740,0.427853,0.863448,0.933617
2011,Redcar and Cleveland,E06000003,-0.408923,1.451283,1,0.001059,0.932628,0.421144,0.835037,0.864340
2011,Stockton-on-Tees,E06000004,1.723617,1.276775,2,0.001672,1.000000,0.318597,0.695618,1.000000
2011,Darlington,E06000005,-0.814488,1.950585,1,0.001059,0.932628,0.421144,0.835037,0.864340
...,...,...,...,...,...,...,...,...,...,...
2021,Hartlepool,E06000001,12.840121,10.120375,4,1.000000,0.362516,0.000000,0.038028,0.230077
2021,Middlesbrough,E06000002,12.196905,8.137314,4,1.000000,0.362516,0.000000,0.038028,0.230077
2021,Redcar and Cleveland,E06000003,12.342235,7.922477,4,1.000000,0.362516,0.000000,0.038028,0.230077
2021,Stockton-on-Tees,E06000004,11.284400,5.482471,1,0.925734,0.375650,0.068201,0.000000,0.455428



[df_tenure_2011_2021_umap] Total missing values: 0
[df_tenure_2011_2021_umap] No missing values found.

[SAVED] The DataFrame has been exported to: Dashboard6\England_Wales_SocioEconomic_Tenure_UMAP_2011_2021.csv (Ready for Tableau Visualization).
